In [1]:
import os
import gdown
import random
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
from pyarrow.parquet import ParquetFile
#from googletrans import Translator
import string
import pyarrow.lib
import ast
import re
import string
import csv
import io
import statistics as statistics
import itertools
import math as math
%matplotlib inline
# set seed for reproducibility
np.random.seed(0) 
from collections import OrderedDict
from nltk.stem.snowball import SnowballStemmer
import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from functional import pseq, seq
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from typing import Union, Tuple
import nltk
from tqdm import tqdm
from unidecode import unidecode
import unidecode

vamos tentar a abordagem do notebook do sherlock (Using Sherlock out-of-the-box) e ir diretamente da Series tirar as features e depois guardar no .parquet.

Acho que e melhor fazer isso pq nao implica tantos problemas partindo de um .parquet inicial

De realçar que a coluna tem de ter sempre nome values

In [2]:
coisa= pd.read_csv('coisa.csv', index_col='ID')

atum=coisa.to_dict('list')
new_v= [i for i in atum.values()]

df = pd.DataFrame(new_v)
df = df.astype(str)

da=df.values.tolist()

dataframe = pd.DataFrame([da])
dataframe=dataframe.T

dataframe.columns=['values']
#dataframe['values']= dataframe['values'].str.lower()
dataframe
data = dataframe.squeeze()
data
#dataframe=dataframe.astype(str)
#dataframe.to_parquet('ds.parquet',compression='snappy',index=None)
#sent=dataframe['values'].tolist()
#sent
#sent = [row]
#sent[:2]
#df = pd.read_parquet('sample.parquet')

#foca= pd.read_parquet('ds.parquet')
#foca
#for lista in sent:
 #   for doc in range(len(lista)):
  #      lista[doc]=lista[doc].lower()
        
#sent
#for lista in sent:
 #   for doc in range(len(lista)):
  #      grande= lista[doc]
#grande=[lista[doc] for doc in range(len(lista)) for lista in sent]
#grande

#title= ' '.join(grande)
#title

0     [351935770576, 351917381362, 351912880805, 351...
1     [A  Araujo, A Baguete Amaro, A Baguete Santos,...
2                                       [A, A, A, A, A]
3           [nan, Baguete, Baguete, Forninha, Forninha]
4            [Araujo, Amaro, Santos, Martinho, Marques]
5              [female, female, female, female, female]
6           [nan, Espinho, Matosinhos, Lisbon, Setubal]
7                         [nan,  None, nan,  None, nan]
8                 [nan,  Portugal, nan,  Portugal, nan]
9           [nan, Espinho, Matosinhos, Lisbon, Setubal]
10                        [nan,  None, nan,  None, nan]
11                [nan,  Portugal, nan,  Portugal, nan]
12    [Single, Single, Married, In a relationship, S...
13    [nan, Profissional Liberal de Informatica, nan...
14    [12:0:00 AM, 12:0:00 AM, 12:0:00 AM, 12:0:00 A...
15                            [nan, nan, nan, nan, nan]
16    [2018-08-28, 2018-09-04, 2017-05-02, 2018-09-0...
Name: values, dtype: object

In [3]:
#tokens = word_tokenize(title)
#palavras=[word.lower() for word in tokens if word.isalpha()]
#stop_words = set(stopwords.words('english','portuguese'))
#palavras= [word for word in palavras if not word in stop_words]
#palavras[:10]

In [4]:
#model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [5]:
#model['lisbon']

o que vou fazer e reproduzir o extract_features do sherlock para coletar as features e estatistics e transforma-lo para um ficheiro csv.

Posteriormente vamos ler esse csv com dtype float32, preenchemos os Nans (depois ver isto) e transformamos para ficheiro parquet.

Este ficheiro parquet vai ser posteriormente usado para seçao de treino do sherlock.

As labels que tinhamos antes da transformaçao da tabela vamos  guardar num ficheiro parquet pq vai ser necessário mais para a frente

In [6]:
coisa2= pd.read_csv('coisa.csv')

atum=coisa2.to_dict('list')
new_ind= [i for i in atum.keys()]

df = pd.DataFrame(new_ind)
#df
df = df.astype(str)
df=df.T

#da=df.values
#da
dataframe2 = pd.DataFrame(df)
dataframe2= dataframe2.T
#dataframe2
dataframe2.columns=['labels']
dataframe2['labels']= dataframe2['labels'].str.lower()

#guardar as labels da coluna para parquet
dataframe2.to_parquet('labels.parquet',compression='snappy',index=None)
#series2 = dataframe2.squeeze()
#series2
foca= pd.read_parquet('labels.parquet')
foca

,labels
0,id
1,nr telefone
2,nome
3,nome proprio
4,meio
5,apelido
6,genero
7,morada
8,cidade
9,pais


vamos fazer agora a extraçao de features com o series

In [7]:
def mode(axis, pre_sorted: bool = False):
    if not pre_sorted:
        axis = sorted(axis)

    _count_max = 1
    _count = 0
    _mode = _current = axis[0]

    for v in axis:
        if v == _current:
            _count = _count + 1
        else:
            if _count > _count_max:
                _count_max = _count
                _mode = _current
            _count = 1
            _current = v

    if _count > _count_max:
        return _current

    return _mode

In [8]:
def prepare_feature_extraction():
    """Download embedding files from Google Drive if they do not exist yet."""
    word_embedding_file = "../treino/glove.6B.50d.txt"
    first_paragraph_vector_file = (
        "../treino/par_vec_trained_400.pkl.docvecs.vectors_docs.npy"
    )
    second_paragraph_vector_file = (
        "../treino/par_vec_trained_400.pkl.trainables.syn1neg.npy"
    )
    third_paragraph_vector_file = (
        "../treino/par_vec_trained_400.pkl.wv.vectors.npy"
    )

    print(
        f"""Preparing feature extraction by downloading 4 files:
        \n {word_embedding_file}, \n {first_paragraph_vector_file},
        \n {second_paragraph_vector_file}, and \n {third_paragraph_vector_file}.
        """
    )

    if not os.path.exists(word_embedding_file):
        print("Downloading GloVe word embedding vectors.")
        file_name = word_embedding_file
        gdown.download(
            url="https://drive.google.com/uc?id=1kayd5oNRQm8-NCvA8pIrtezbQ-B1_Vmk",
            output=file_name,
        )

        print("GloVe word embedding vectors were downloaded.")

    if not os.path.exists(first_paragraph_vector_file):
        print("Downloading first paragraph vector file.")
        file_name = first_paragraph_vector_file
        gdown.download(
            url="https://drive.google.com/uc?id=1vdyGJ4aB71FCaNqJKYX387eVufcH4SAu",
            output=file_name,
        )

    if not os.path.exists(second_paragraph_vector_file):
        print("Downloading second paragraph vector file.")
        file_name = second_paragraph_vector_file
        gdown.download(
            url="https://drive.google.com/uc?id=1hwE8We05oZLrACRibY8jc81NGughv79q",
            output=file_name,
        )

        print("Downloaded second paragraph vector file.")

    if not os.path.exists(third_paragraph_vector_file):
        print("Downloading third paragraph vector file.")
        file_name = third_paragraph_vector_file
        gdown.download(
            url="https://drive.google.com/uc?id=1StGoalk5SMbWX8Z-5weSbIAtK771UwoC",
            output=file_name,
        )

        print("Downloaded third paragraph vector file.")

    print("All files for extracting word and paragraph embeddings are present.")

    word_to_embedding = {}


def initialise_word_embeddings():
    

    global word_to_embedding

    print("Initialising word embeddings")
    word_vectors_f = open('../treino/glove.6B.50d.txt', encoding='utf-8')

    for w in word_vectors_f:
        term, vector = w.strip().split(' ', 1)
        vector = list(map(float, vector.split(' ')))

        word_to_embedding[term] = vector

    word_vectors_f.close()
    
    
DIM = 400
model: Doc2Vec
def initialise_pretrained_model(dim):
    
    global model

    filename = f"../treino/par_vec_trained_{dim}.pkl"

    assert dim == DIM

    model = Doc2Vec.load(filename)
    model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    print(
        f"Initialise Doc2Vec Model, {dim} dim (filename = {filename})"
    )
    
    STOPWORDS_ENGLISH = None


def initialise_nltk():
    

    nltk.download("punkt")
    nltk.download("stopwords")

    global STOPWORDS_ENGLISH

    STOPWORDS_ENGLISH = stopwords.words("english","portuguese")

    print(f"Initialised NLTK")
    
    


In [9]:
_is_first = True


def is_first() -> bool:
    return _is_first


def set_first():
    global _is_first
    _is_first = False


def reset_first():
    global _is_first
    _is_first = True

### Bag of Characters

In [10]:
#vai ser necessário para verificar carateres
CHARACTERS_TO_CHECK = (
        [c for c in string.printable if c not in ('\n', '\v', '\r', '\t', '\\', '^')] + ['\\', '^']
)

#vai calcular as estatisticas
def compute_stats(values):
    _min = min(values)
    _max = max(values)
    _sum = sum(values)

    _mean = np.mean(values)

    x = values - _mean

    _variance = np.mean(x * x)

    if _variance == 0:
        _skew = 0
        _kurtosis = -3
    else:
        _skew = np.mean(x ** 3) / _variance ** 1.5
        _kurtosis = np.mean(x ** 4) / _variance ** 2 - 3

    return _mean, _variance, _skew, _kurtosis, _min, _max, _sum



# Input: a single column in the form of Python list
# Output: ordered dictionary holding bag of character features
def extract_bag_of_characters_features(col_values: list, features: OrderedDict):
    # Create a set of unique chars from the string vectors to quickly test whether to perform expensive
    # processing for any given char
    char_set = set(''.join(col_values))

    for c in CHARACTERS_TO_CHECK:
        value_feature_name = f'n_[{c}]'

        if c in char_set:
            counts = [s.count(c) for s in col_values]

            has_any = any(counts)
        else:
            has_any = False

        if has_any:
            _any = 1
            _all = 1 if all(counts) else 0
            _mean, _variance, _skew, _kurtosis, _min, _max, _sum = compute_stats(counts)
            _median = statistics.median(counts)

            if is_first():
                # the first output needs fully expanded keys (to drive CSV header)
                features[value_feature_name + '-agg-any'] = _any
                features[value_feature_name + '-agg-all'] = _all
                features[value_feature_name + '-agg-mean'] = _mean
                features[value_feature_name + '-agg-var'] = _variance
                features[value_feature_name + '-agg-min'] = _min
                features[value_feature_name + '-agg-max'] = _max
                features[value_feature_name + '-agg-median'] = _median
                features[value_feature_name + '-agg-sum'] = _sum
                features[value_feature_name + '-agg-kurtosis'] = _kurtosis
                features[value_feature_name + '-agg-skewness'] = _skew
            else:
                # subsequent lines only care about values, so we can pre-render a block of CSV. This
                # cuts overhead of storing granular values in the features dictionary
                features[value_feature_name + '-pre-rendered'] = \
                    f'{_any},{_all},{_mean},{_variance},{_min},{_max},{_median},{_sum},{_kurtosis},{_skew}'
        else:
            if is_first():
                # the first output needs fully expanded keys (to drive CSV header)
                features[value_feature_name + '-agg-any'] = 0
                features[value_feature_name + '-agg-all'] = 0
                features[value_feature_name + '-agg-mean'] = 0
                features[value_feature_name + '-agg-var'] = 0
                features[value_feature_name + '-agg-min'] = 0
                features[value_feature_name + '-agg-max'] = 0
                features[value_feature_name + '-agg-median'] = 0
                features[value_feature_name + '-agg-sum'] = 0
                features[value_feature_name + '-agg-kurtosis'] = -3
                features[value_feature_name + '-agg-skewness'] = 0
            else:
                # assign pre-rendered defaults
                features[value_feature_name + '-pre-rendered'] = '0,0,0,0,0,0,0,0,-3,0'


### Word Embeddings

A minha duvida de momento é se nos devemos basear no documento que o sherlock fornece no word embeddings

In [11]:
word_to_embedding={}

    
ZEROS = [0] * 50

# nans for mean, median, stdev and mode for each embedding
NANS = ','.join(map(str, [np.nan] * 50 * 4))


def transpose(a):
    # transpose array:
    #   >>> theArray = [['a','b','c'],['d','e','f'],['g','h','i']]
    #   >>> [*zip(*theArray)]
    #   [('a', 'd', 'g'), ('b', 'e', 'h'), ('c', 'f', 'i')]
    #
    #   https://stackoverflow.com/questions/4937491/matrix-transpose-in-python
    return [*zip(*a)]

# Input: a single column in the form of a Python list
# Output: ordered dictionary holding word embedding features
def extract_word_embeddings_features(col_values: list, features: OrderedDict):
    num_embeddings = 50

    embeddings = []

    global word_to_embedding

    if not word_to_embedding:
        initialise_word_embeddings()

    for col_value in map(str.lower, col_values):
        if col_value in word_to_embedding:
            embeddings.append(word_to_embedding.get(col_value))
        else:
            embeddings_to_all_words = [word_to_embedding.get(w) for w in col_value.split(' ') if w in word_to_embedding]

            n = len(embeddings_to_all_words)

            if n == 1:
                embeddings.append(embeddings_to_all_words[0])
            elif n > 1:
                mean_of_word_embeddings = np.mean(embeddings_to_all_words, dtype=float, axis=0)
                embeddings.append(mean_of_word_embeddings)

    n_rows = len(embeddings)

    if n_rows == 0:
        if is_first():
            # the first output needs fully expanded keys (to drive CSV header)
            # need to maintain same insertion order as the other case, hence running for loop per feature
            for i in range(num_embeddings):
                features['word_embedding_avg_' + str(i)] = np.nan
            for i in range(num_embeddings):
                features['word_embedding_std_' + str(i)] = np.nan
            for i in range(num_embeddings):
                features['word_embedding_med_' + str(i)] = np.nan
            for i in range(num_embeddings):
                features['word_embedding_mode_' + str(i)] = np.nan
        else:
            # subsequent lines only care about values, so we can pre-render a block of CSV. This
            # cuts overhead of storing granular values in the features dictionary
            features['word_embedding-pre-rendered'] = NANS

        features['word_embedding_feature'] = 0

    else:
        if n_rows > 1:
            mean_embeddings = []
            std_embeddings = []
            med_embeddings = []
            mode_embeddings = []

            # transpose array (if using numpy, stats would operate on axis=0):
            for axis0 in transpose(embeddings):
                # mode requires sorted list, and Python's sort is super quick on presorted arrays. The subsequent
                # median calc also calls sorted(), so this helps there, too.
                #
                # Re: sorting in Python
                #   https://stackoverflow.com/questions/1436962/python-sort-method-on-list-vs-builtin-sorted-function
                axis0 = sorted(axis0)

                _mean = sum(axis0) / n_rows

                mean_embeddings.append(_mean)

                _variance = sum((x - _mean) ** 2 for x in axis0) / n_rows
                std_embeddings.append(math.sqrt(_variance))

                med_embeddings.append(statistics.median(axis0))

                mode_embeddings.append(mode(axis0, True))
        # n_rows == 1
        else:
            # if only one dimension, then mean, median and mode are equivalent to the embedding data
            mean_embeddings = med_embeddings = mode_embeddings = embeddings[0]
            std_embeddings = ZEROS

        if is_first():
            # the first output needs fully expanded keys (to drive CSV header)
            for i, e in enumerate(mean_embeddings):
                features['word_embedding_avg_' + str(i)] = e
            for i, e in enumerate(std_embeddings):
                features['word_embedding_std_' + str(i)] = e
            for i, e in enumerate(med_embeddings):
                features['word_embedding_med_' + str(i)] = e
            for i, e in enumerate(mode_embeddings):
                features['word_embedding_mode_' + str(i)] = e
        else:
            # subsequent lines only care about values, so we can pre-render a block of CSV. This
            # cuts overhead of storing granular values in the features dictionary
            features['word_embedding-pre-rendered'] = \
                ','.join(map(lambda x: '%g' % x,
                             itertools.chain(mean_embeddings, std_embeddings, med_embeddings, mode_embeddings)))

        features['word_embedding_feature'] = 1

### Bag of words

In [12]:
def escape_for_regex(c):
    if c in ('[', ']', '\\', '^', '-'):
        return '\\' + c
    else:
        return c
    
def count_pattern_in_cells(values: list, pat):
    return [len(re.findall(pat, s)) for s in values]


def count_pattern_in_cells_with_non_zero_count(values: list, pat):
    cell_counts = [len(re.findall(pat, s)) for s in values]

    return sum(1 for c in cell_counts if c > 0), cell_counts


NUMBER_PATTERN = re.compile(r"[0-9]")
TEXT_PATTERN = re.compile(r"[a-zA-Z]")
WORD_PATTERN = re.compile(r"[\w+]")


# SPECIAL_CHARACTERS_REGEX = '[' + ''.join(escape_for_regex(c) for c in string.printable
#                                          if c not in ('\n', '\f', '\r', '\t')
#                                          and not re.match(r'[a-zA-Z0-9\s]', c)) + ']'
#
# SPECIAL_CHARACTERS_PATTERN = re.compile(SPECIAL_CHARACTERS_REGEX)

SPECIAL_CHARACTERS_PATTERN = re.compile(r'[!@#$%^&*(),.?":{}|<>]')


# Input: a single column in the form of a Python list
# Output: ordered dictionary holding bag of words features
def extract_bag_of_words_features(col_values: list, features: OrderedDict, n_val):
    if not n_val:
        return

    # Entropy of column
    freq_dist = nltk.FreqDist(col_values)
    probs = [freq_dist.freq(l) for l in freq_dist]
    features["col_entropy"] = -sum(p * math.log(p, 2) for p in probs)

    # Fraction of cells with unique content
    num_unique = len(set(col_values))
    features["frac_unique"] = num_unique / n_val

    # Fraction of cells with numeric content -> frac text cells doesn't add information
    numeric_cell_nz_count, numeric_char_counts = count_pattern_in_cells_with_non_zero_count(
        col_values, NUMBER_PATTERN
    )
    text_cell_nz_count, text_char_counts = count_pattern_in_cells_with_non_zero_count(
        col_values, TEXT_PATTERN
    )

    features["frac_numcells"] = numeric_cell_nz_count / n_val
    features["frac_textcells"] = text_cell_nz_count / n_val

    # Average + std number of numeric tokens in cells
    features["avg_num_cells"] = np.mean(numeric_char_counts)
    features["std_num_cells"] = np.std(numeric_char_counts)

    # Average + std number of textual tokens in cells
    features["avg_text_cells"] = np.mean(text_char_counts)
    features["std_text_cells"] = np.std(text_char_counts)

    # Average + std number of special characters in each cell
    spec_char_counts = count_pattern_in_cells(col_values, SPECIAL_CHARACTERS_PATTERN)

    features["avg_spec_cells"] = np.mean(spec_char_counts)
    features["std_spec_cells"] = np.std(spec_char_counts)

    # Average number of words in each cell
    word_counts = count_pattern_in_cells(col_values, WORD_PATTERN)

    features["avg_word_cells"] = np.mean(word_counts)
    features["std_word_cells"] = np.std(word_counts)

    features["n_values"] = n_val

    lengths = [len(s) for s in col_values]
    n_none = sum(1 for _l in lengths if _l == 0)

    has_any = any(lengths)

    if has_any:
        _any = 1
        _all = 1 if all(lengths) else 0
        _mean, _variance, _skew, _kurtosis, _min, _max, _sum = compute_stats(lengths)
        _median = statistics.median(lengths)

        if is_first():
            # the first output needs fully expanded keys (to drive CSV header)
            features["length-agg-any"] = _any
            features["length-agg-all"] = _all
            features["length-agg-mean"] = _mean
            features["length-agg-var"] = _variance
            features["length-agg-min"] = _min
            features["length-agg-max"] = _max
            features["length-agg-median"] = _median
            features["length-agg-sum"] = _sum
            features["length-agg-kurtosis"] = _kurtosis
            features["length-agg-skewness"] = _skew
        else:
            # subsequent lines only care about values, so we can pre-render a block of CSV. This
            # cuts overhead of storing granular values in the features dictionary
            features[
                "length-pre-rendered"
            ] = f"{_any},{_all},{_mean},{_variance},{_min},{_max},{_median},{_sum},{_kurtosis},{_skew}"
    else:
        if is_first():
            features["length-agg-any"] = 0
            features["length-agg-all"] = 0
            features["length-agg-mean"] = 0
            features["length-agg-var"] = 0
            features["length-agg-min"] = 0
            features["length-agg-max"] = 0
            features["length-agg-median"] = 0
            features["length-agg-sum"] = 0
            features["length-agg-kurtosis"] = -3
            features["length-agg-skewness"] = 0
        else:
            # assign pre-rendered defaults
            features["length-pre-rendered"] = "0,0,0,0,0,0,0,0,-3,0"

    features["none-agg-has"] = 1 if n_none > 0 else 0
    features["none-agg-percent"] = n_none / n_val
    features["none-agg-num"] = n_none
    features["none-agg-all"] = 1 if n_none == n_val else 0

### Infer paragrgaph embedding

In [13]:
def tokenise(values):
    joined = " ".join(s for s in values if len(s) >= 2)

    # stopwords need apostrophe
    filtered = "".join(
        e for e in joined if e.isalnum() or e.isspace() or e == "'"
    ).lower()

    return [
        word
        for word in nltk.word_tokenize(filtered)
        if len(word) >= 2 and word not in STOPWORDS_ENGLISH
    ]



# Input: a single column in the form of a pandas Series.
# Output: ordered dictionary holding paragraph vector features
def infer_paragraph_embeddings_features(
    col_values: list, features: OrderedDict, dim, reuse_model
):
    if not reuse_model or model is None:
        # Load pretrained paragraph vector model
        initialise_pretrained_model(dim)

    # Resetting the random seed before inference keeps the inference vectors deterministic. Gensim uses random values
    # in the inference process, so setting the seed just beforehand makes the inference repeatable.
    # https://github.com/RaRe-Technologies/gensim/issues/447

    # To make the inference repeatable across runtime launches, we also need to set PYTHONHASHSEED
    # prior to launching the execution environment (i.e. jupyter notebook).  E.g. export PYTHONHASHSEED=13
    # See above Github thread for more information.
    model.random.seed(13)

    tokens = tokenise(col_values)

    # Infer paragraph vector for data sample.
    inferred = model.infer_vector(tokens, steps=20, alpha=0.025)

    if is_first():
        # the first output needs fully expanded keys (to drive CSV header)
        for idx, value in enumerate(inferred):
            features["par_vec_" + str(idx)] = value
    else:
        # subsequent lines only care about values, so we can pre-render a block of CSV. This
        # cuts overhead of storing granular values in the features dictionary
        features["par_vec-pre-rendered"] = ",".join(map(lambda x: "%g" % x, inferred))

### Extrair as features e tratar do csv final das estatisticas

In [14]:
def keys_to_csv(keys):
    """
    Encode a list of strings into an Excel CSV compatible header.
    Wraps all items with double quotes to prevent legitimate values containing a comma from being interpreted as a
    separator, and encodes existing double quotes with two double quotes.
    """
    with io.StringIO() as output:
        writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
        writer.writerow(keys)

        return output.getvalue()


def extract_features(output_file, data: Union[pd.DataFrame, pd.Series]):
    
    vec_dim = 400
    reuse_model = True
    verify_keys = False

    first_keys = None

    reset_first()

    #Enquanto o ficheiro esta aberto
    #tqdm é uma barra de progresso de extraçao
    with open(output_file, "w") as outfile:
        for raw_sample in tqdm(data, desc="A extrair Features"):
            
            #A limitar as samples a pelo menos 1000
            n_samples = 1000
            n_values = len(raw_sample)

            if n_samples > n_values:
                n_samples = n_values

            random.seed(13)
            raw_sample = random.sample(raw_sample, k=n_samples)
            
            #criamos um dicionario
            features = OrderedDict()
            
            #extraçao através de vários metodos
            extract_bag_of_characters_features(raw_sample, features)
            extract_word_embeddings_features(raw_sample, features)
            extract_bag_of_words_features(raw_sample, features, n_samples)

            # TODO use data_no_null version?
            infer_paragraph_embeddings_features(
                raw_sample, features, vec_dim, reuse_model
            )
            
            #buscar as keys das features
            if first_keys is None:
                first_keys = features.keys()
                print(f"Exporting {len(first_keys)} column features")

                first_keys_str = keys_to_csv(features.keys())

                outfile.write(first_keys_str + "\n")

                set_first()
            elif verify_keys:
                keys = ",".join(features.keys())
                if first_keys_str != keys:
                    key_list = list(features.keys())

                    print(
                        f"keys are NOT equal. k1 len={len(first_keys)}, k2 len={len(keys)}"
                    )

                    for idx, k1 in enumerate(first_keys):
                        k2 = key_list[idx]

                        if k1 != k2:
                            print(f"{k1} != {k2}")

            outfile.write(",".join(map(str, features.values())) + "\n")

In [15]:
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 4 files:
        
 ../treino/glove.6B.50d.txt, 
 ../treino/par_vec_trained_400.pkl.docvecs.vectors_docs.npy,
        
 ../treino/par_vec_trained_400.pkl.trainables.syn1neg.npy, and 
 ../treino/par_vec_trained_400.pkl.wv.vectors.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Doc2Vec Model, 400 dim (filename = ../treino/par_vec_trained_400.pkl)
Initialised NLTK


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utilizador\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Utilizador\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
extract_features(
    "../treino/fet.csv",
    data
)
feature_vectors = pd.read_csv("../treino/fet.csv", dtype=np.float32)

A extrair Features:   0%|                                                                       | 0/17 [00:00<?, ?it/s]

Exporting 1588 column features


A extrair Features: 100%|█████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 226.67it/s]


In [17]:
feature_vectors

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.8,0.56,0.0,2.0,1.0,4.0,-1.153061,0.343622,...,-0.000902,-0.000282,-0.000650,0.000532,0.000401,-0.000291,-0.000471,-0.000608,0.000384,-0.000193
1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.103248,-0.063492,0.006317,0.117746,-0.146321,0.007247,0.044775,-0.092377,0.037838,0.001763
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.000190,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.034420,0.007949,-0.029827,-0.033962,-0.053325,-0.029663,-0.001368,-0.043288,0.031743,-0.048483
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.102316,-0.064254,0.006304,0.118812,-0.145438,0.008002,0.044849,-0.093132,0.037887,0.000884
5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.070782,0.009367,-0.029513,0.002591,-0.028669,-0.005073,-0.009544,-0.064592,0.003442,-0.081422
6,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.096801,0.081798,-0.036092,0.049728,-0.039897,0.023525,0.011557,0.033269,0.074097,-0.125004
7,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.151323,0.042758,-0.039135,-0.027172,-0.089891,-0.097401,0.029705,-0.071213,-0.016445,-0.137992
8,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.104860,0.032809,-0.048400,0.000831,-0.107931,-0.057476,-0.007998,-0.110144,0.002657,-0.163869
9,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.096801,0.081798,-0.036092,0.049728,-0.039897,0.023525,0.011557,0.033269,0.074097,-0.125004


In [18]:
subs_mean= pd.DataFrame(feature_vectors.mean()).transpose()

feature_vectors.fillna(subs_mean.iloc[0],inplace=True)
subs_mean=None
feature_vectors
#subs_mean


,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.8,0.56,0.0,2.0,1.0,4.0,-1.153061,0.343622,...,-0.000902,-0.000282,-0.000650,0.000532,0.000401,-0.000291,-0.000471,-0.000608,0.000384,-0.000193
1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.103248,-0.063492,0.006317,0.117746,-0.146321,0.007247,0.044775,-0.092377,0.037838,0.001763
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.000190,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.034420,0.007949,-0.029827,-0.033962,-0.053325,-0.029663,-0.001368,-0.043288,0.031743,-0.048483
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.102316,-0.064254,0.006304,0.118812,-0.145438,0.008002,0.044849,-0.093132,0.037887,0.000884
5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.070782,0.009367,-0.029513,0.002591,-0.028669,-0.005073,-0.009544,-0.064592,0.003442,-0.081422
6,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.096801,0.081798,-0.036092,0.049728,-0.039897,0.023525,0.011557,0.033269,0.074097,-0.125004
7,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.151323,0.042758,-0.039135,-0.027172,-0.089891,-0.097401,0.029705,-0.071213,-0.016445,-0.137992
8,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.104860,0.032809,-0.048400,0.000831,-0.107931,-0.057476,-0.007998,-0.110144,0.002657,-0.163869
9,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.096801,0.081798,-0.036092,0.049728,-0.039897,0.023525,0.011557,0.033269,0.074097,-0.125004


In [19]:
feature_vectors.to_parquet('../treino/fet.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


In [20]:
spike_cols = [col for col in feature_vectors.columns if '.' in col]
spike_cols

['n_[.]-agg-any',
 'n_[.]-agg-all',
 'n_[.]-agg-mean',
 'n_[.]-agg-var',
 'n_[.]-agg-min',
 'n_[.]-agg-max',
 'n_[.]-agg-median',
 'n_[.]-agg-sum',
 'n_[.]-agg-kurtosis',
 'n_[.]-agg-skewness']

-----------------------------------------------------------------------

### Antibioticos

In [21]:
antibioticos= pd.read_csv('anti_values.csv')
antibioticos = antibioticos.squeeze()
antibioticos

0    ['2015-01', '2015-01', '2015-01', '2015-01', '...
1    ['alentejo', 'centro', 'centro', 'centro', 'ce...
2    ['unidade local de saude do baixo alentejo, e....
3    ['38.014123, -7.8721227', '39.7414812, -8.7982...
4    ['grupo c', 'grupo c', 'grupo c', 'grupo d', '...
5    ['9162', '19973', '20778', '25669', '1720', '4...
6    ['343630', '231541', '359202', '478722', '2375...
7    ['2.666236980159815', '8.626118052526333', '5....
Name: values, dtype: object

In [22]:
extract_features(
    "../treino/anti_stats.csv",
    antibioticos
)
feature_vectors_anti = pd.read_csv("../treino/anti_stats.csv", dtype=np.float32)

A extrair Features:   0%|                                                                        | 0/8 [00:00<?, ?it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 23.95it/s]


In [23]:
feature_vectors_anti

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.205,0.162975,0.0,1.0,0.0,205.0,0.135910,1.461475,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.045,0.042975,0.0,1.0,0.0,45.0,17.269342,4.389686,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.029,0.028159,0.0,1.0,0.0,29.0,29.512625,5.613611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.040,0.038400,0.0,1.0,0.0,40.0,20.041666,4.694855,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.101,0.090799,0.0,1.0,0.0,101.0,5.013337,2.648271,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [24]:
feature_vectors_anti.to_parquet('../treino/fet_anti.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------------------------------------------------

### Certificados de Obito por instituição de saude

In [25]:
obit= pd.read_csv('cert_obit.csv')
obit = obit.squeeze()
obit

0    ['2014-01', '2014-01', '2014-01', '2014-01', '...
1    ['unidade de saude da ilha de sao jorge - cent...
2    ['4', '4', '1', '117', '72', '25', '112', '30'...
Name: values, dtype: object

In [26]:
extract_features(
    "../treino/cert_obit_stats.csv",
    obit
)
feature_vectors_cert_obit = pd.read_csv("../treino/cert_obit_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 23.25it/s]

Exporting 1588 column features


In [27]:
feature_vectors_cert_obit

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.187,0.152031,0.0,1.0,0.0,187.0,0.577606,1.605492,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.007,0.006951,0.0,1.0,0.0,7.0,137.864197,11.826419,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [28]:
feature_vectors_cert_obit.to_parquet('../treino/fet_cert_obit.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------------------------------------------------

### Colheira de orgaos anual

In [29]:
col_org= pd.read_csv('col_org.csv')
col_org = col_org.squeeze()
col_org

0    ['2015', '2013', '2017', '2014', '2015', '2018...
1    ['coracao', 'rim', 'pancreas', 'pancreas', 'ri...
2    ['51', '539', '29', '30', '586', '58', '30', '...
Name: values, dtype: object

In [30]:
extract_features(
    "../treino/col_org_stats.csv",
    col_org
)
feature_vectors_col_org = pd.read_csv("../treino/col_org_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 73.18it/s]

Exporting 1588 column features


In [31]:
feature_vectors_col_org

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.12500,0.109375,0.0,1.0,0.0,40.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.03125,0.030273,0.0,1.0,0.0,8.0,27.032259,5.388159,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [32]:
feature_vectors_col_org.to_parquet('../treino/fet_col_org.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


---------------------------------------------------------

### Dadores de Orgãos

In [33]:
dad_org= pd.read_csv('dad_org.csv')
dad_org = dad_org.squeeze()
dad_org

0    ['2018', '2011', '2012', '2017', '2013', '2015...
1    ['416.0', '375.0', '317.0', '440.0', '359.0', ...
2    ['344.0', '301.0', '252.0', '351.0', '295.0', ...
3    ['72.0', '74.0', '65.0', '89.0', '64.0', '82.0...
4    ['10.29', '10.56', '10.42', '10.31', '10.49', ...
5    ['40.42', '35.5113636364', '30.4222648752', '4...
Name: values, dtype: object

In [34]:
extract_features(
    "../treino/dad_org_stats.csv",
    dad_org
)
feature_vectors_dad_org = pd.read_csv("../treino/dad_org_stats.csv", dtype=np.float32)

A extrair Features: 100%|███████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 146.35it/s]

Exporting 1588 column features


In [35]:
feature_vectors_dad_org

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.125000,0.109375,0.0,1.0,0.0,8.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.138889,0.119599,0.0,1.0,0.0,10.0,2.361290,2.088370,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.125000,0.109375,0.0,1.0,0.0,9.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.125000,0.109375,0.0,1.0,0.0,8.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.111111,0.098765,0.0,1.0,0.0,8.0,4.125000,2.474874,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.050847,0.048262,0.0,1.0,0.0,6.0,14.720238,4.089039,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [36]:
feature_vectors_dad_org.to_parquet('../treino/fet_dad_org.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------------------------------------------------

### Dadores de Sangue

In [37]:
dad_sang= pd.read_csv('dad_sang.csv')
dad_sang = dad_sang.squeeze()
dad_sang

0    ['2010-12', '2010-12', '2010-12', '2010-12', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['hospital espirito santo de evora, epe', 'uni...
3    ['38.5684861,-7.9032848', '39.3003425,-7.42638...
4    ['4548.0', '5910.0', '3214.0', '11235.0', '400...
5    ['1210.0', '722.0', '901.0', '1080.0', '785.0'...
6    ['4541.0', '5188.0', '1135.0', '10155.0', '322...
Name: values, dtype: object

In [38]:
extract_features(
    "../treino/dad_sang_stats.csv",
    dad_sang
)
feature_vectors_dad_sang = pd.read_csv("../treino/dad_sang_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 23.81it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 26.62it/s]


In [39]:
feature_vectors_dad_sang

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.170,0.141100,0.0,1.0,0.0,170.0,1.087172,1.757035,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.126,0.110124,0.0,1.0,0.0,126.0,3.080673,2.254035,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.108,0.096336,0.0,1.0,0.0,108.0,4.380335,2.525933,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.107,0.095551,0.0,1.0,0.0,107.0,4.465615,2.542758,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [40]:
feature_vectors_dad_sang.to_parquet('../treino/fet_dad_sang.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


--------------------------------------------------------------

###  Dadores vivos e falecidos

In [41]:
dad_vf= pd.read_csv('dad_vf.csv')
dad_vf = dad_vf.squeeze()
dad_vf

0    ['2012-06', '2012-09', '2012-12', '2014-06', '...
1    ['89', '74', '76', '101', '83', '101', '98', '...
2    ['8.5', '7.1', '7.3', '9.7', '8.0', '9.8', '9....
Name: values, dtype: object

In [42]:
extract_features(
    "../treino/dad_vf_stats.csv",
    dad_vf
)
feature_vectors_dad_vf = pd.read_csv("../treino/dad_vf_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 63.78it/s]

Exporting 1588 column features


In [43]:
feature_vectors_dad_vf

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.167355,0.139348,0.0,1.0,0.0,81.0,1.176301,1.782218,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.049822,0.047340,0.0,1.0,0.0,14.0,15.123863,4.138099,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.019560,0.019177,0.0,1.0,0.0,8.0,46.144951,6.938656,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [44]:
feature_vectors_dad_vf.to_parquet('../treino/fet_dad_vf.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------------------------------

### Doenças de Declaraçao obrigatoria

In [45]:
ddc_ob= pd.read_csv('ddc_ob.csv')
ddc_ob = ddc_ob.squeeze()
ddc_ob

0    ['2016-11', '2016-12', '2016-12', '2016-12', '...
1    ['regiao de saude lvt', 'regiao de saude do ce...
2    ['area metropolitana de lisboa', 'regiao de co...
3    ['gonorreia', 'doenca dos legionarios', 'salmo...
4    ['feminino', 'feminino', 'feminino', 'masculin...
5    ['35-44 anos', '45-54 anos', '<1 ano', '35-44 ...
6    ['1', '1', '1', '1', '1', '1', '1', '1', '1', ...
Name: values, dtype: object

In [46]:
extract_features(
    "../treino/ddc_ob_stats.csv",
    ddc_ob
)
feature_vectors_ddc_ob = pd.read_csv("../treino/ddc_ob_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 24.19it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 26.02it/s]


In [47]:
feature_vectors_ddc_ob

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.176,0.145024,0.0,1.0,0.0,176.0,0.89541,1.701591,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.00000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.00000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.00000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.00000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.00000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.00000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [48]:
feature_vectors_ddc_ob.to_parquet('../treino/fet_ddc_ob.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------------------------------

### Episodios de internamento com dispositivos medicos

In [49]:
ep_int= pd.read_csv('ep_int.csv')
ep_int = ep_int.squeeze()
ep_int

0    ['2016-12', '2016-02', '2016-01', '2016-04', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['unidade local de saude do norte alentejo, ep...
3    ['39.3003425,-7.4263845', '38.6554005,-9.05923...
4    ['7022', '2234', '1398', '6920', '825', '1002'...
5    ['13', '31', '89', '385', '5', '0', '46', '0',...
Name: values, dtype: object

In [50]:
extract_features(
    "../treino/ep_int_stats.csv",
    ep_int
)
feature_vectors_ep_int = pd.read_csv("../treino/ep_int_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 27.78it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 28.57it/s]


In [51]:
feature_vectors_ep_int

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.161,0.135079,0.0,1.0,0.0,161.0,1.403075,1.844743,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.046,0.043884,0.0,1.0,0.0,46.0,16.787348,4.334437,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.062,0.058156,0.0,1.0,0.0,62.0,11.195130,3.632510,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.043,0.041151,0.0,1.0,0.0,43.0,18.300747,4.505635,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [52]:
feature_vectors_ep_int.to_parquet('../treino/fet_ep_int.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------------------

### Frequencia de casos confirmados de gripe

In [53]:
casos_g= pd.read_csv('casos_g.csv')
casos_g = casos_g.squeeze()
casos_g

0     ['2012', '2005', '2013', '2003', '2014', '2015...
1     ['678', '800', '1262', '695', '868', '903', '4...
2     ['a(h3)', 'a(h3)', 'b/a(h1)pdm09', 'b', 'a(h1)...
3     ['39.2', '53.8', '43.5', '32.5', '54.0', '55.0...
4     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
5     ['0.0', '1.4', '0.0', '3.1', '0.0', '0.0', '0....
6     ['0.0', '0.0', '42.3', '0.0', '59.7', '4.2', '...
7     ['97.7', '81.9', '6.4', '1.3', '38.8', '30.0',...
8     ['0.0', '16.7', '0.0', '95.6', '0.0', '0.0', '...
9     ['2.3', '0.0', '49.5', '0.0', '1.3', '65.8', '...
10    ['0.0', '0.0', '1.8', '0.0', '0.2', '0.0', '0....
Name: values, dtype: object

In [54]:
extract_features(
    "../treino/casos_g_stats.csv",
    casos_g
)
feature_vectors_casos_g = pd.read_csv("../treino/casos_g_stats.csv", dtype=np.float32)

A extrair Features: 100%|█████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 122.23it/s]

Exporting 1588 column features


In [55]:
feature_vectors_casos_g

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.201389,0.160831,0.0,1.0,0.0,29.0,0.217691,1.489191,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.061538,0.057751,0.0,1.0,0.0,8.0,11.315574,3.649051,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.025641,0.024984,0.0,1.0,0.0,5.0,34.026318,6.002192,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.076389,0.070554,0.0,1.0,0.0,11.0,8.173615,3.189611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.335664,0.222994,0.0,1.0,0.0,48.0,-1.515570,0.696010,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.167939,0.139735,0.0,1.0,0.0,22.0,1.156380,1.776621,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.190840,0.154420,0.0,1.0,0.0,25.0,0.475849,1.573483,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.088235,0.080450,0.0,1.0,0.0,12.0,6.430108,2.903465,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.157895,0.132964,0.0,1.0,0.0,21.0,1.520833,1.876388,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.203125,0.161865,0.0,1.0,0.0,26.0,0.177979,1.475798,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [56]:
feature_vectors_casos_g.to_parquet('../treino/fet_casos_g.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-------------------------------------------------------

### Incidencia semanal de sindrome gripal

In [57]:
inc_sp= pd.read_csv('inc_sp.csv')
inc_sp = inc_sp.squeeze()
inc_sp

0    ['1999', '1999', '1999', '1999', '2000', '2000...
1    ['40', '42', '47', '52', '1', '3', '40', '41',...
2    ['4.7', '4.4', '12.5', '16.1', '13.0', '52.5',...
Name: values, dtype: object

In [58]:
extract_features(
    "../treino/inc_sp_stats.csv",
    inc_sp
)
feature_vectors_inc_sp = pd.read_csv("../treino/inc_sp_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 28.04it/s]

Exporting 1588 column features


In [59]:
feature_vectors_inc_sp

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.191,0.154519,0.0,1.0,0.0,191.0,0.471696,1.572163,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.025,0.024375,0.0,1.0,0.0,25.0,35.025642,6.084870,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.090,0.081900,0.0,1.0,0.0,90.0,6.210012,2.865312,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [60]:
feature_vectors_inc_sp.to_parquet('../treino/fet_inc_sp.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-------------------------------------------------------

### Movimento clinico de utentes com problemas de comportamentos aditivos e dependentes

In [61]:
mcu= pd.read_csv('mcu.csv')
mcu = mcu.squeeze()
mcu

0    ['2019-01', '2019-01', '2019-01', '2016-07', '...
1    ['problemas ligados ao alcool', 'problemas lig...
2    ['318', '191', '212', '1493', '1705', '2037', ...
3    ['104', '162', '46', '100', '866', '1106', '62...
4    ['16627', '36883', '3769', '23938', '389893', ...
5    ['5126.0', '13182.0', '1765.0', '3900.0', '248...
6    ['27.33', '34.02', '51.3', '26.68', '32.8', '3...
7    ['18.25', '10.54', '10.89', '17.54', '10.64', ...
Name: values, dtype: object

In [62]:
extract_features(
    "../treino/mcu_stats.csv",
    mcu
)
feature_vectors_mcu = pd.read_csv("../treino/mcu_stats.csv", dtype=np.float32)

A extrair Features:  38%|████████████████████████                                        | 3/8 [00:00<00:00, 29.12it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 27.97it/s]


In [63]:
feature_vectors_mcu

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.169,0.140439,0.0,1.0,0.0,169.0,1.120529,1.766502,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.034,0.032844,0.0,1.0,0.0,34.0,24.446960,5.142661,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.051,0.048399,0.0,1.0,0.0,51.0,14.661584,4.081861,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.138,0.118956,0.0,1.0,0.0,138.0,2.406470,2.099159,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.024,0.023424,0.0,1.0,0.0,24.0,36.691257,6.220230,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.047,0.044791,0.0,1.0,0.0,47.0,16.325914,4.280878,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [64]:
feature_vectors_mcu.to_parquet('../treino/fet_mcu.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


---------------------------------------

### Rastreios Oncologicos

Nao percebo porque deu erro aqui

ras_onc= pd.read_csv('ras_onc.csv')
ras_onc = ras_onc.squeeze()
ras_onc


extract_features(
    "../treino/ras_onc_stats.csv",
    ras_onc
)
feature_vectors_ras_onc = pd.read_csv("../treino/ras_onc.csv")
feature_vectors_ras_onc= feature_vectors_ras_onc.astype(float)
#feature_vectors_ras_onc=pd.to_numeric(feature_vectors_ras_onc.columns, errors='coerce')

feature_vectors_ras_onc

-----------------------------------------------------------------

### Registo Nacional de anomalias geneticas

In [65]:
an_gen= pd.read_csv('an_gen.csv')
an_gen = an_gen.squeeze()
an_gen

0     ['2001', '2004', '2011', '2006', '2008', '2007...
1     ['38.7643755,-9.1644375', '38.7643755,-9.16443...
2     ['7.824074074069999', '6.5274151436', '7.69230...
3     ['5.601851851849999', '4.8366013071900005', '5...
4     ['24.444444444400002', '26.405228758200003', '...
5     ['1.2962962963', '1.8954248366', '2.0833333333...
6     ['3.93518518519', '3.8562091503300007', '2.938...
7     ['4.33579335793', '6.53594771242', '4.86111111...
8     ['6.01851851852', '3.98692810458', '5.07478632...
9     ['14.027777777799999', '12.614379085', '11.111...
10    ['19.7222222222', '21.4379084967', '17.3611111...
11    ['3.92066420664', '3.78590078329', '3.15170940...
12    ['9.08671586716', '8.23529411765', '12.3931623...
Name: values, dtype: object

In [66]:
extract_features(
    "../treino/an_gen_stats.csv",
    an_gen
)
feature_vectors_an_gen = pd.read_csv("../treino/an_gen_stats.csv", dtype=np.float32)

A extrair Features:  62%|██████████████████████████████████████▊                        | 8/13 [00:00<00:00, 72.73it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 69.52it/s]


In [67]:
feature_vectors_an_gen

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.203947,0.162353,0.0,1.0,0.0,31.0,0.159424,1.469498,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.073446,0.068052,0.0,1.0,0.0,26.0,8.694653,3.270268,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.102857,0.092278,0.0,1.0,0.0,36.0,4.836872,2.614741,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.119565,0.105269,0.0,1.0,0.0,44.0,3.499439,2.345088,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.098315,0.088649,0.0,1.0,0.0,35.0,5.280463,2.698233,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.101408,0.091125,0.0,1.0,0.0,36.0,4.973964,2.640826,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.124294,0.108845,0.0,1.0,0.0,44.0,3.187390,2.277584,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.115942,0.102499,0.0,1.0,0.0,40.0,3.756148,2.399197,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.076705,0.070821,0.0,1.0,0.0,27.0,8.120114,3.181213,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [68]:
feature_vectors_an_gen.to_parquet('../treino/fet_an_gen.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


----------------------------------------------------------------

### Reservas

In [69]:
res= pd.read_csv('res.csv')
res = res.squeeze()
res

0    ['2010-12', '2010-12', '2010-12', '2010-12', '...
1    ['regiao de saude lvt', 'regiao de saude do ce...
2    ['centro hospitalar universitario lisboa centr...
3    ['38.7166948,-9.1371248', '40.516364,-8.082743...
4    ['total', 'total', 'total', 'total', 'total', ...
5    ['558.0', '5127.0', '134.7579694314287', '249....
Name: values, dtype: object

In [70]:
extract_features(
    "../treino/res_stats.csv",
    res
)
feature_vectors_res = pd.read_csv("../treino/res_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 25.20it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 27.15it/s]


In [71]:
feature_vectors_res

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.163,0.136431,0.0,1.0,0.0,163.0,1.329713,1.824750,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.053,0.050191,0.0,1.0,0.0,53.0,13.923891,3.990475,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.057,0.053751,0.0,1.0,0.0,57.0,12.604305,3.821558,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [72]:
feature_vectors_res.to_parquet('../treino/fet_res.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


--------------------------------------

### Saude da mulher e da criança

In [73]:
sa_mc= pd.read_csv('sa_mc.csv')
sa_mc = sa_mc.squeeze()
sa_mc

0    ['2014-01', '2014-01', '2014-01', '2014-01', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['aces alentejo litoral', 'aces sao mamede', '...
3    ['38.3841359,-8.5132031', '39.2967086,-7.42847...
4    ['60', '86', '289', '76', '15', '199', '152', ...
5    ['73.17073', '78.18182', '76.8617', '76.0', '6...
6    ['27', '14', '55', '5', '0', '32', '36', '27',...
7    ['36.0', '13.86139', '15.068489999999999', '5....
Name: values, dtype: object

In [74]:
extract_features(
    "../treino/sa_mc_stats.csv",
    sa_mc
)
feature_vectors_sa_mc = pd.read_csv("../treino/sa_mc_stats.csv", dtype=np.float32)

A extrair Features:  38%|████████████████████████                                        | 3/8 [00:00<00:00, 26.09it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 27.78it/s]


In [75]:
feature_vectors_sa_mc

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.200,0.160000,0.0,1.0,0.0,200.0,0.250000,1.500000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.045,0.042975,0.0,1.0,0.0,45.0,17.269342,4.389686,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.036,0.034704,0.0,1.0,0.0,36.0,22.815123,4.981478,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.097,0.087591,0.0,1.0,0.0,97.0,5.416698,2.723361,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.029,0.028159,0.0,1.0,0.0,29.0,29.512625,5.613611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.120,0.105600,0.0,1.0,0.0,120.0,3.469697,2.338738,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [76]:
feature_vectors_sa_mc.to_parquet('../treino/fet_sa_mc.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------------------

### Taxa de cobertura da vacina antigripal sazonal na população em portugal

In [77]:
tax_c= pd.read_csv('tax_c.csv')
tax_c = tax_c.squeeze()
tax_c

0    ['2005', '2006', '2009', '2004', '2016', '2018...
1    ['38.7643755,-9.1644375', '38.7643755,-9.16443...
2    ['15.4', '19.1', '18.3', '18.4', '16.2', '21.0...
Name: values, dtype: object

In [78]:
extract_features(
    "../treino/tax_c_stats.csv",
    tax_c
)
feature_vectors_tax_c = pd.read_csv("../treino/tax_c_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 78.95it/s]

Exporting 1588 column features


In [79]:
feature_vectors_tax_c

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.190789,0.154389,0.0,1.0,0.0,29.0,0.477152,1.573897,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.036145,0.034838,0.0,1.0,0.0,6.0,22.704166,4.970329,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [80]:
feature_vectors_tax_c.to_parquet('../treino/tax_c_.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


----------------------------------------------------

### Taxa de incidencia de diabetes

In [81]:
tax_d= pd.read_csv('tax_d.csv')
tax_d = tax_d.squeeze()
tax_d

0    ['2003.0', '2004.0', '2009.0', '2010.0', '2018...
1    ['414.7', '525.3', '606.9', '780.1', '580.6542...
2    ['315.5', '449.9', '538.2', '568.1', '627.1307...
3    ['362.9', '485.9', '571.1', '669.2', '605.0', ...
Name: values, dtype: object

In [82]:
extract_features(
    "../treino/tax_d_stats.csv",
    tax_d
)
feature_vectors_tax_d = pd.read_csv("../treino/tax_d_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 76.93it/s]

Exporting 1588 column features


In [83]:
feature_vectors_tax_d

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.208000,0.164736,0.0,1.0,0.0,52.0,0.070319,1.438860,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.037344,0.035950,0.0,1.0,0.0,9.0,21.816570,4.880222,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.024896,0.024276,0.0,1.0,0.0,6.0,35.192200,6.098541,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.042194,0.040414,0.0,1.0,0.0,10.0,18.744053,4.554564,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [84]:
feature_vectors_tax_d.to_parquet('../treino/tax_d_.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------------------------

### Transplantes

In [85]:
tansp= pd.read_csv('tansp.csv')
tansp = tansp.squeeze()
tansp

0    ['2011', '2013', '2013', '2016', '2017', '2017...
1    ['pulmao', 'coracao', 'figado', 'figado', 'cor...
2    ['31', '20', '166', '108', '44', '58', '24', '...
3    ['18', '55', '243', '272', '46', '34', '26', '...
Name: values, dtype: object

In [86]:
extract_features(
    "../treino/tansp_stats.csv",
    tansp
)
feature_vectors_tansp = pd.read_csv("../treino/tansp_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 81.64it/s]

Exporting 1588 column features


In [87]:
feature_vectors_tansp

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.12500,0.109375,0.0,1.0,0.0,35.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.03913,0.037599,0.0,1.0,0.0,9.0,20.596279,4.753554,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.03125,0.030273,0.0,1.0,0.0,7.0,27.032259,5.388159,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [88]:
feature_vectors_tansp.to_parquet('../treino/tansp_.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


--------------------------------------------

### Vacinaçao

In [89]:
vac= pd.read_csv('vac.csv')
vac = vac.squeeze()
vac

0     ['2015-07', '2015-08', '2015-08', '2015-09', '...
1     ['regiao de saude norte', 'regiao de saude nor...
2     ['aces douro ii - douro sul', 'aces alto tras-...
3     ['41.0953745,-7.8123805', '41.8069684,-6.75879...
4     ['409', '706', '613', '1100', '1106', '357', '...
5     ['93.59268', '91.56939', '92.31928', '92.98394...
6     ['558', '878', '843', '1369', '1368', '504', '...
7     ['93.93939', '92.13011999999999', '96.23288000...
8     ['693', '1002', '979', '1295', '1298', '431', ...
9     ['89.30412', '91.09091', '90.73216', '84.41982...
10    ['145', '240', '176', '443', '13405', '3426', ...
11    ['0.70038', '0.57008', '0.6090800000000001', '...
Name: values, dtype: object

In [90]:
extract_features(
    "../treino/vac_stats.csv",
    vac
)
feature_vectors_vac = pd.read_csv("../treino/vac_stats.csv", dtype=np.float32)

A extrair Features:  25%|███████████████▊                                               | 3/12 [00:00<00:00, 25.20it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 28.44it/s]


In [91]:
feature_vectors_vac

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.177,0.145671,0.0,1.0,0.0,177.0,0.864784,1.692567,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.044,0.042064,0.0,1.0,0.0,44.0,17.773298,4.446718,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.038,0.036556,0.0,1.0,0.0,38.0,21.355291,4.832731,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.079,0.072759,0.0,1.0,0.0,79.0,7.744004,3.121539,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.044,0.042064,0.0,1.0,0.0,44.0,17.773298,4.446718,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.087,0.079431,0.0,1.0,0.0,87.0,6.589543,2.930792,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.039,0.037479,0.0,1.0,0.0,39.0,20.681608,4.762521,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.108,0.096336,0.0,1.0,0.0,108.0,4.380335,2.525933,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [92]:
feature_vectors_vac.to_parquet('../treino/vac.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


---------------------------------------

### Antibioticos carbapenemes

In [93]:
ant_ca= pd.read_csv('ant_ca.csv')
ant_ca= ant_ca.squeeze()
ant_ca

0    ['2015-01', '2015-01', '2015-01', '2015-01', '...
1    ['alentejo', 'alentejo', 'centro', 'centro', '...
2    ['unidade local de saude do baixo alentejo, e....
3    ['38.014123,-7.8721227', '39.3003425,-7.426384...
4    ['grupo c', 'grupo c', 'grupo c', 'grupo b', '...
5    ['46', '262', '884', '225', '423', '1830', '47...
6    ['3767', '5686', '8991', '3132', '4618', '2607...
7    ['1.21', '4.42', '8.95', '6.7', '8.4', '6.56',...
Name: values, dtype: object

In [94]:
extract_features(
    "../treino/ant_ca_stats.csv",
    ant_ca
)
feature_vectors_ant_ca = pd.read_csv("../treino/ant_ca_stats.csv", dtype=np.float32)

A extrair Features:  38%|████████████████████████                                        | 3/8 [00:00<00:00, 27.78it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 29.19it/s]


In [95]:
feature_vectors_ant_ca

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.203,0.161791,0.0,1.0,0.0,203.0,0.180814,1.476758,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.054,0.051084,0.0,1.0,0.0,54.0,13.575601,3.946594,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.038,0.036556,0.0,1.0,0.0,38.0,21.355291,4.832731,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.059,0.055519,0.0,1.0,0.0,59.0,12.011852,3.743241,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.132,0.114576,0.0,1.0,0.0,132.0,2.727831,2.174358,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [96]:
feature_vectors_ant_ca.to_parquet('../treino/ant_ca.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------

### Antibioticos cefalosporinas

In [97]:
ant_ce= pd.read_csv('ant_ce.csv')
ant_ce= ant_ce.squeeze()
ant_ce

0    ['2015-01', '2015-01', '2015-01', '2015-01', '...
1    ['alentejo', 'alentejo', 'centro', 'centro', '...
2    ['unidade local de saude do baixo alentejo, e....
3    ['38.014123,-7.8721227', '38.0162314,-8.695676...
4    ['grupo c', 'grupo b', 'grupo b', 'grupo f', '...
5    ['500', '163', '511', '713', '541', '1646', '2...
6    ['3313', '2188', '2846', '1141', '2186', '6488...
7    ['13.12', '6.96', '15.23', '38.45', '19.84', '...
Name: values, dtype: object

In [98]:
extract_features(
    "../treino/ant_ce_stats.csv",
    ant_ce
)
feature_vectors_ant_ce = pd.read_csv("../treino/ant_ce_stats.csv", dtype=np.float32)

A extrair Features:  38%|████████████████████████                                        | 3/8 [00:00<00:00, 28.84it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 29.96it/s]


In [99]:
feature_vectors_ant_ce

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.210,0.165900,0.0,1.0,0.0,210.0,0.027728,1.423983,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.050,0.047500,0.0,1.0,0.0,50.0,15.052631,4.129483,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.038,0.036556,0.0,1.0,0.0,38.0,21.355291,4.832731,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.028,0.027216,0.0,1.0,0.0,28.0,30.743092,5.722158,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.098,0.088396,0.0,1.0,0.0,98.0,5.312729,2.704206,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [100]:
feature_vectors_ant_ce.to_parquet('../treino/ant_ce.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


----------------------------

### Certificado de obito por instituiçao de saude (RETIRADO)

cert_obi= pd.read_csv('cert_obi.csv')
cert_obi= cert_obi.squeeze()
cert_obi

extract_features(
    "../treino/cert_obi_stats.csv",
    cert_obi
)
feature_vectors_cert_obi = pd.read_csv("../treino/cert_obi_stats.csv", dtype=np.float32)

feature_vectors_cert_obi

feature_vectors_cert_obi.to_parquet('../treino/cert_obi.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

-----------------------------------

### dadores vivos e falecidos(retirado)

dad_vef= pd.read_csv('dad_vef.csv')
dad_vef= dad_vef.squeeze()
dad_vef

extract_features(
    "../treino/dad_vef_stats.csv",
    dad_vef
)
feature_vectors_dad_vef = pd.read_csv("../treino/dad_vef_stats.csv", dtype=np.float32)

feature_vectors_dad_vef

feature_vectors_dad_vef.to_parquet('../treino/dad_vef.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

---------------------------

### Dsipositivos medicos por nomenclatura portuguesa do despositivo medico

In [101]:
dm_dm= pd.read_csv('dm_dm.csv')
dm_dm= dm_dm.squeeze()
dm_dm

0    ['2016-11', '2016-11', '2016-11', '2016-11', '...
1    ['regiao de saude lvt', 'regiao de saude lvt',...
2    ['hospital vila franca de xira', 'hospital vil...
3    ['38.952565,-8.992398', '38.952565,-8.992398',...
4    ['componente femoral bicompartimental cimentad...
5    ['51', '310', '3', '23', '75', '3', '12', '335...
Name: values, dtype: object

In [102]:
extract_features(
    "../treino/dm_dm_stats.csv",
    dm_dm
)
feature_vectors_dm_dm = pd.read_csv("../treino/dm_dm_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 27.27it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 25.53it/s]


In [103]:
feature_vectors_dm_dm

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.172,0.142416,0.0,1.0,0.0,172.0,1.021683,1.738299,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.030,0.029100,0.0,1.0,0.0,30.0,28.364262,5.510377,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.001,0.000999,0.0,1.0,0.0,1.0,995.000977,31.575323,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.020,0.019600,0.0,1.0,0.0,20.0,45.020409,6.857143,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [104]:
feature_vectors_dm_dm.to_parquet('../treino/dm_dm.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------------------

### Dissuasao da toxicodependencia

In [105]:
diss_t= pd.read_csv('diss_t.csv')
diss_t= diss_t.squeeze()
diss_t

0    ['2019-01', '2012-12', '2013-12', '2016-06', '...
1    ['1139', '9046', '9316', '5389', '8503', '9370...
2    ['46', '942', '722', '608', '805', '863', '265...
3    ['496', '4908', '4952', '3799', '5837', '6350'...
Name: values, dtype: object

In [106]:
extract_features(
    "../treino/diss_t_stats.csv",
    diss_t
)
feature_vectors_diss_t = pd.read_csv("../treino/diss_t_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 43.01it/s]

Exporting 1588 column features


In [107]:
feature_vectors_diss_t

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.177156,0.145772,0.0,1.0,0.0,152.0,0.860034,1.691164,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.052298,0.049563,0.0,1.0,0.0,33.0,14.176396,4.021989,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.037244,0.035857,0.0,1.0,0.0,20.0,21.888685,4.887605,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.033003,0.031914,0.0,1.0,0.0,20.0,25.334129,5.228205,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [108]:
feature_vectors_diss_t.to_parquet('../treino/diss_t.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------------

### episodios de internamento com dispositivos medicos(retirado)


ep_interdm= pd.read_csv('ep_interdm.csv')
ep_interdm= ep_interdm.squeeze()
ep_interdm

extract_features(
    "../treino/ep_interdm_stats.csv",
    ep_interdm
)
feature_vectors_ep_interdm = pd.read_csv("../treino/ep_interdm_stats.csv", dtype=np.float32)

feature_vectors_ep_interdm

feature_vectors_ep_interdm.to_parquet('../treino/ep_interdm.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

-----------------------------

### evolucao diaria do indice icaro

In [109]:

ev_ic= pd.read_csv('ev_ic.csv')
ev_ic= ev_ic.squeeze()
ev_ic

0    ['2019-05-08', '2019-05-13', '2019-05-15', '20...
1    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
Name: values, dtype: object

In [110]:
extract_features(
    "../treino/ev_ic_stats.csv",
    ev_ic
)
feature_vectors_ev_ic = pd.read_csv("../treino/ev_ic_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 30.29it/s]

Exporting 1588 column features


In [111]:
feature_vectors_ev_ic

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.208,0.164736,0.0,1.0,0.0,208.0,0.070319,1.438860,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.267,0.195711,0.0,1.0,0.0,267.0,-0.890425,1.053364,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [112]:
feature_vectors_ev_ic.to_parquet('../treino/ep_interdm.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


----------------------------

### frequencia de casos confirmados de gripe(retirado)


freq_cg= pd.read_csv('freq_cg.csv')
freq_cg= freq_cg.squeeze()
freq_cg

extract_features(
    "../treino/freq_cg_stats.csv",
    freq_cg
)
feature_vectors_freq_cg = pd.read_csv("../treino/freq_cg_stats.csv", dtype=np.float32)

feature_vectors_freq_cg

feature_vectors_freq_cg.to_parquet('../treino/freq_cg.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

---------------------------------

### frequencia de casos de sindrome gripal devido a infecao por outros virus respira

In [113]:

freq_csg= pd.read_csv('freq_csg.csv')
freq_csg= freq_csg.squeeze()
freq_csg

0    ['2012.0', '2015.0', '2013.0', '2014.0', '2016...
1        ['678.0', '405.0', '709.0', '402.0', '658.0']
2        ['101.0', '152.0', '242.0', '202.0', '265.0']
3             ['50.5', '48.3', '43.8', '35.1', '38.0']
4             ['17.8', '16.4', '28.9', '14.9', '13.0']
5    ['19.03333333333333', '11.2', '19.033333333333...
6                  ['3.0', '1.0', '7.0', '1.5', '1.0']
7                ['15.8', '13.8', '5.4', '8.4', '5.0']
8               ['10.6', '8.0', '6.6', '17.8', '10.0']
9                 ['12.9', '1.3', '8.3', '5.4', '4.0']
Name: values, dtype: object

In [114]:
extract_features(
    "../treino/freq_csg_stats.csv",
    freq_csg
)
feature_vectors_freq_csg = pd.read_csv("../treino/freq_csg_stats.csv", dtype=np.float32)

A extrair Features: 100%|█████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 185.17it/s]

Exporting 1588 column features


In [115]:
feature_vectors_freq_csg

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.200000,0.160000,0.0,1.0,0.0,10.0,0.250000,1.500000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.177778,0.146173,0.0,1.0,0.0,8.0,0.841216,1.685591,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.155556,0.131358,0.0,1.0,0.0,7.0,1.612782,1.900732,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.050000,0.047500,0.0,1.0,0.0,2.0,15.052631,4.129483,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.025000,0.024375,0.0,1.0,0.0,1.0,35.025642,6.084870,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.045455,0.043388,0.0,1.0,0.0,3.0,17.047619,4.364358,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.114286,0.101224,0.0,1.0,0.0,4.0,3.879032,2.424672,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.027027,0.026297,0.0,1.0,0.0,1.0,32.027779,5.833333,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.105263,0.094183,0.0,1.0,0.0,4.0,4.617647,2.572479,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.027778,0.027006,0.0,1.0,0.0,1.0,31.028572,5.747049,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [116]:
feature_vectors_freq_csg.to_parquet('../treino/freq_csg.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


---------------------------

### incidencia semanal de sindrome gripal(retirado)


inc_sg= pd.read_csv('inc_sg.csv')
inc_sg= inc_sg.squeeze()
inc_sg

extract_features(
    "../treino/inc_sg_stats.csv",
    inc_sg
)
feature_vectors_inc_sg = pd.read_csv("../treino/inc_sg_stats.csv", dtype=np.float32)

feature_vectors_inc_sg

feature_vectors_inc_sg.to_parquet('../treino/inc_sg.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

--------------------------------------

### mapeamento entre codigos de centros de custo usados no rhv e codigos da hierquia

In [117]:

map_ccu= pd.read_csv('map_ccu.csv')
map_ccu= map_ccu.squeeze()
map_ccu

0     ['2018-06', '2018-06', '2018-06', '2018-06', '...
1     ['1', '1', '1', '1', '1', '1', '1', '1', '1', ...
2     ['111223081', '1113101102', '1113101103', '111...
3     ['cdp santo tirso/trofa', 'usf renascer', 'usf...
4     ['arsn', 'arsn', 'arsn', 'arsn', 'arsn', 'arsn...
5     ['administracao regional saude norte,  i.p.', ...
6     ['ars', 'ars', 'ars', 'ars', 'ars', 'ars', 'ar...
7     ['h2', 'h2', 'h2', 'h2', 'h2', 'h2', 'h2', 'h2...
8     ['1132630', '1131371', '1131372', '1131373', '...
9     ['cdp santo tirso / trofa', 'usf renascer', 'u...
10    ['20070208.0', '20061003.0', '20061016.0', '20...
11    ['cdp', 'usf-b', 'usf-b', 'usf-a', 'usf-b', 'u...
12    ['centro diag. pneum.', 'usf modelo b', 'usf m...
13    ['csp', 'csp', 'csp', 'csp', 'csp', 'csp', 'cs...
14    ['aces ou cs - cuidados de saude primarios', '...
15    ['uf', 'uf', 'uf', 'uf', 'uf', 'uf', 'uf', 'uf...
16    ['acesso a propria unidade funcional ou servic...
17    ['10021.0', '10029.0', '10029.0', '10029.0

In [118]:
extract_features(
    "../treino/map_ccu_stats.csv",
    map_ccu
)
feature_vectors_map_ccu = pd.read_csv("../treino/map_ccu_stats.csv", dtype=np.float32)

A extrair Features:  11%|███████                                                        | 3/27 [00:00<00:00, 29.99it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 28.97it/s]


In [119]:
feature_vectors_map_ccu

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.175,0.144375,0.0,1.0,0.0,175.0,0.926407,1.710674,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.084,0.076944,0.0,1.0,0.0,84.0,6.996465,2.999411,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.142,0.121836,0.0,1.0,0.0,142.0,2.207755,2.051281,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.328,0.220416,0.0,1.0,0.0,328.0,-1.463124,0.732718,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [120]:
feature_vectors_map_ccu.to_parquet('../treino/map_ccu.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-------------------------------------

### monitorizaçao ambiental de legionella

In [121]:

mal= pd.read_csv('mal.csv')
mal= mal.squeeze()
mal

0    ['2013', '2017', '2015', '2016', '2018', '2019...
1    ['38.7643755,-9.1644375', '38.7643755,-9.16443...
2    ['507', '2216', '1475', '1877', '2919', '3352'...
3    ['38', '527', '245', '379', '489', '557', '137...
4    ['28', '397', '210', '320', '333', '486', '109...
Name: values, dtype: object

In [122]:
extract_features(
    "../treino/mal_stats.csv",
    mal
)
feature_vectors_mal = pd.read_csv("../treino/mal_stats.csv", dtype=np.float32)

A extrair Features: 100%|███████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 151.37it/s]

Exporting 1588 column features


In [123]:
feature_vectors_mal

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.140625,0.120850,0.0,1.0,0.0,9.0,2.274747,2.067546,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.016129,0.015869,0.0,1.0,0.0,1.0,57.016392,7.682213,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.054545,0.051570,0.0,1.0,0.0,3.0,13.391026,3.923140,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [124]:
feature_vectors_mal.to_parquet('../treino/mal.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------

### noticias de nascimento digital e registo de nascer de utentes

In [125]:

nnd_rn= pd.read_csv('nnd_rn.csv')
nnd_rn= nnd_rn.squeeze()
nnd_rn

0    ['2019-01', '2019-01', '2019-01', '2019-01', '...
1    ['regiao de saude do centro', 'regiao de saude...
2    ['centro hospitalar universitario cova da beir...
3    ['40.2804158,-7.4922407', '39.7414812,-8.79827...
4    ['28', '144', '390', '212', '104', '124', '84'...
5    ['0.0', '150.0', '341.0', '114.0', '61.0', '12...
Name: values, dtype: object

In [126]:
extract_features(
    "../treino/nnd_rn_stats.csv",
    nnd_rn
)
feature_vectors_nnd_rn = pd.read_csv("../treino/nnd_rn_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 26.54it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 29.13it/s]


In [127]:
feature_vectors_nnd_rn

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.200,0.160000,0.0,1.0,0.0,200.0,0.250000,1.500000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.058,0.054636,0.0,1.0,0.0,58.0,12.302951,3.781924,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.029,0.028159,0.0,1.0,0.0,29.0,29.512625,5.613611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.172,0.142416,0.0,1.0,0.0,172.0,1.021683,1.738299,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [128]:
feature_vectors_nnd_rn.to_parquet('../treino/nnd_rn.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


--------------------------

### partos e cesarianas

In [129]:

pc= pd.read_csv('pc.csv')
pc= pc.squeeze()
pc

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do centro', 'regiao de saude...
2    ['centro hospitalar e universitario de coimbra...
3    ['40.2191523,-8.4098211', '39.4054629,-9.12584...
4    ['405', '142', '70', '206', '86', '185', '135'...
5    ['120', '50', '16', '57', '25', '64', '67', '1...
Name: values, dtype: object

In [130]:
extract_features(
    "../treino/pc_stats.csv",
    pc
)
feature_vectors_pc = pd.read_csv("../treino/pc_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 28.30it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 29.85it/s]


In [131]:
feature_vectors_pc

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.190,0.153900,0.0,1.0,0.0,190.0,0.497726,1.580420,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.045,0.042975,0.0,1.0,0.0,45.0,17.269342,4.389686,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.035,0.033775,0.0,1.0,0.0,35.0,23.607698,5.060405,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.029,0.028159,0.0,1.0,0.0,29.0,29.512625,5.613611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [132]:
feature_vectors_pc.to_parquet('../treino/pc.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------

### registo de testamentos vitais

In [133]:

rtv= pd.read_csv('rtv.csv')
rtv= rtv.squeeze()
rtv

0     ['2014-07', '2014-07', '2014-07', '2014-07', '...
1     ['acores', 'alentejo', 'centro', 'lisboa e val...
2     ['usi corvo', 'uls litoral alentejano', 'aces ...
3     ['39.701736,-31.105386', '38.0162314,-8.695676...
4     ['0.0', '1.0', '0.0', '11.0', '1.0', '0.0', '0...
5     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
6     ['0', '1', '0', '11', '1', '0', '0', '0', '0',...
7     ['0', '1', '0', '11', '0', '0', '0', '0', '0',...
8     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
9     ['0', '0', '0', '10', '0', '0', '0', '0', '0',...
10    ['0', '0', '0', '0', '0', '0', '0', '0', '0', ...
Name: values, dtype: object

In [134]:
extract_features(
    "../treino/rtv_stats.csv",
    rtv
)
feature_vectors_rtv = pd.read_csv("../treino/rtv_stats.csv", dtype=np.float32)

A extrair Features:  27%|█████████████████▏                                             | 3/11 [00:00<00:00, 27.26it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 30.81it/s]


In [135]:
feature_vectors_rtv

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.169,0.140439,0.0,1.0,0.0,169.0,1.120529,1.766502,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.046,0.043884,0.0,1.0,0.0,46.0,16.787348,4.334437,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.184,0.150144,0.0,1.0,0.0,184.0,0.660273,1.631034,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.263,0.193831,0.0,1.0,0.0,263.0,-0.840867,1.076631,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.082,0.075276,0.0,1.0,0.0,82.0,7.284447,3.047039,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.079,0.072759,0.0,1.0,0.0,79.0,7.744004,3.121539,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.207,0.164151,0.0,1.0,0.0,207.0,0.091952,1.446358,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.098,0.088396,0.0,1.0,0.0,98.0,5.312729,2.704206,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [136]:
feature_vectors_rtv.to_parquet('../treino/rtv.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------------------

### taxa incidencia de diabetes(retirado)


tinc_d= pd.read_csv('tinc_d.csv')
tinc_d= tinc_d.squeeze()
tinc_d

extract_features(
    "../treino/tinc_d_stats.csv",
    tinc_d
)
feature_vectors_tinc_d = pd.read_csv("../treino/tinc_d_stats.csv", dtype=np.float32)

feature_vectors_tinc_d

feature_vectors_tinc_d.to_parquet('../treino/tinc_d.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

--------------------

### transplantes(retirado)


trp= pd.read_csv('trp.csv')
trp= trp.squeeze()
trp

extract_features(
    "../treino/trp_stats.csv",
    trp
)
feature_vectors_trp = pd.read_csv("../treino/trp_stats.csv", dtype=np.float32)

feature_vectors_trp

feature_vectors_trp.to_parquet('../treino/trp.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

------------------------

### vacinaçao(retirado)


va= pd.read_csv('va.csv')
va= va.squeeze()
va

extract_features(
    "../treino/va_stats.csv",
    va
)
feature_vectors_va = pd.read_csv("../treino/va_stats.csv", dtype=np.float32)

feature_vectors_va

feature_vectors_va.to_parquet('../treino/va.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

----------------------

### Amostras

In [137]:

amos= pd.read_csv('amos.csv')
amos= amos.squeeze()
amos

0    ['26-02-2020', '27-02-2020', '28-02-2020', '29...
1    ['0.0', '0.0', '0.0', '0.0', '50.0', '115.0', ...
Name: values, dtype: object

In [138]:
extract_features(
    "../treino/amos_stats.csv",
    amos
)
feature_vectors_amos = pd.read_csv("../treino/amos_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 26.67it/s]

Exporting 1588 column features


In [139]:
feature_vectors_amos

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.176,0.145024,0.0,1.0,0.0,176.0,0.895410,1.701591,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.131,0.113839,0.0,1.0,0.0,131.0,2.784336,2.187312,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [140]:
feature_vectors_amos.to_parquet('../treino/amos.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


-----------------

### data

In [141]:

datas= pd.read_csv('datas.csv')
datas= datas.squeeze()
datas

0     ['26-02-2020', '27-02-2020', '28-02-2020', '29...
1     ['norte', 'centro', 'lvt', 'alentejo', 'algarv...
2     ['0.0', '0.0', '0.0', '0.0', '0.0', '2.0', '4....
3     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
4     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
5     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
6     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
7     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '2....
8     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
9     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
10    ['25.0', '51.0', '59.0', '70.0', '85.0', '85.0...
11    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
12    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
Name: values, dtype: object

In [142]:
extract_features(
    "../treino/datas_stats.csv",
    datas
)
feature_vectors_datas = pd.read_csv("../treino/datas_stats.csv", dtype=np.float32)

A extrair Features:  31%|███████████████████▍                                           | 4/13 [00:00<00:00, 31.01it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 30.44it/s]


In [143]:
feature_vectors_datas

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.176,0.145024,0.0,1.0,0.0,176.0,0.895410,1.701591,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.133,0.115311,0.0,1.0,0.0,133.0,2.672199,2.161527,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.125,0.109375,0.0,1.0,0.0,125.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.123,0.107871,0.0,1.0,0.0,123.0,3.270332,2.295720,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.096,0.086784,0.0,1.0,0.0,96.0,5.522861,2.742784,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.137,0.118231,0.0,1.0,0.0,137.0,2.458019,2.111402,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.105,0.093975,0.0,1.0,0.0,105.0,4.641128,2.577039,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.194,0.156364,0.0,1.0,0.0,194.0,0.395334,1.547687,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.105,0.093975,0.0,1.0,0.0,105.0,4.641128,2.577039,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [144]:
feature_vectors_datas.to_parquet('../treino/datas.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


---------------------

### datas concelhos

In [145]:

data_c= pd.read_csv('data_c.csv')
data_c= data_c.squeeze()
data_c

0    ['24-03-2020', '25-03-2020', '26-03-2020', '27...
1    ['abrantes', 'aguiar da beira', 'alandroal', '...
2    ['2574.0', '2902.0', '4880.0', '5888.0', '7932...
Name: values, dtype: object

In [146]:
extract_features(
    "../treino/data_c_stats.csv",
    data_c
)
feature_vectors_data_c = pd.read_csv("../treino/data_c_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 27.88it/s]

Exporting 1588 column features


In [147]:
feature_vectors_data_c

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.238,0.181356,0.0,1.0,0.0,238.0,-0.485983,1.230454,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.131,0.113839,0.0,1.0,0.0,131.0,2.784336,2.187312,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [148]:
feature_vectors_data_c.to_parquet('../treino/data_c.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------

### RT

In [149]:
rt= pd.read_csv('rt.csv')
rt= rt.squeeze()
rt

0    ['2020-02-21', '2020-02-22', '2020-02-23', '20...
1    ['norte', 'centro', 'lvt', 'alentejo', 'algarv...
2    ['1.46308073117461', '1.49136236727605', '1.61...
Name: values, dtype: object

In [150]:
extract_features(
    "../treino/rt_stats.csv",
    rt
)
feature_vectors_rt = pd.read_csv("../treino/rt_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 29.68it/s]

Exporting 1588 column features


In [151]:
feature_vectors_rt 

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.193,0.155751,0.0,1.0,0.0,193.0,0.420505,1.555797,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.107,0.095551,0.0,1.0,0.0,107.0,4.465615,2.542758,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [152]:
feature_vectors_rt.to_parquet('../treino/rt.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


------------------------------

### vacinas

In [153]:
vaci= pd.read_csv('vaci.csv')
vaci= vaci.squeeze()
vaci

0     ['27-12-2020', '28-12-2020', '29-12-2020', '30...
1     ['9926.0', '10545.0', '32982.0', '40718.0', '3...
2     ['9926.0', '10545.0', '32982.0', '40718.0', '3...
3     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
4     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
5     ['9926.0', '10545.0', '32982.0', '40718.0', '3...
6     ['9926.0', '10545.0', '32982.0', '40718.0', '3...
7     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
8     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
9     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
10    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
11    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
Name: values, dtype: object

In [154]:
extract_features(
    "../treino/vaci_stats.csv",
    vaci
)
feature_vectors_vaci = pd.read_csv("../treino/vaci_stats.csv", dtype=np.float32)

A extrair Features:  25%|███████████████▊                                               | 3/12 [00:00<00:00, 27.26it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 28.23it/s]


In [155]:
feature_vectors_vaci

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.155,0.130975,0.0,1.0,0.0,155.0,1.635045,1.906579,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.117,0.103311,0.0,1.0,0.0,117.0,3.679511,2.383173,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.102,0.091596,0.0,1.0,0.0,102.0,4.917507,2.630116,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.107,0.095551,0.0,1.0,0.0,107.0,4.465615,2.542758,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.142,0.121836,0.0,1.0,0.0,142.0,2.207755,2.051281,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.140,0.120400,0.0,1.0,0.0,140.0,2.305648,2.075006,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.117,0.103311,0.0,1.0,0.0,117.0,3.679511,2.383173,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.212,0.167056,0.0,1.0,0.0,212.0,-0.013983,1.409261,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.224,0.173824,0.0,1.0,0.0,224.0,-0.247054,1.323988,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.251,0.187999,0.0,1.0,0.0,251.0,-0.680823,1.148554,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [156]:
feature_vectors_vaci.to_parquet('../treino/vaci.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


----------------------------------

### vacinas detalhe

In [157]:
va_det= pd.read_csv('va_det.csv')
va_det= va_det.squeeze()
va_det

0     ['11-01-2021', '18-01-2021', '25-01-2021', '01...
1     ['norte', 'centro', 'lvt', 'alentejo', 'algarv...
2     ['651900.0', '651900.0', '651900.0', '651900.0...
3     ['571981.0', '571981.0', '571981.0', '571981.0...
4     ['134391', '188118', '447628', '468552', '5041...
5     ['134140', '185537', '380618', '342390', '3487...
6     ['247', '2576', '66994', '126130', '155466', '...
7     ['4', '5', '16', '32', '25', '41', '46', '53',...
8     ['0.008221480500000001', '0.0130757054', '0.02...
9     ['1.2756199999999998e-05', '0.0001308479', '0....
10    ['84951', '133908', '223551', '219708', '20629...
11    ['138', '1414', '34429', '80488', '118172', '2...
12    ['85089', '135322', '257980', '300196', '32447...
13    ['0.0082094973', '0.012940605400000001', '0.02...
14    ['1.3335999999999999e-05', '0.0001366462000000...
15    ['0.008222833399999999', '0.0130772516', '0.02...
16    ['10347893.0', '10347892.5', '10347892.5', '10...
Name: values, dtype: object

In [158]:
extract_features(
    "../treino/va_det_stats.csv",
    va_det
)
feature_vectors_va_det = pd.read_csv("../treino/va_det_stats.csv", dtype=np.float32)

A extrair Features:  29%|██████████████████▌                                            | 5/17 [00:00<00:00, 49.50it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 45.69it/s]


In [159]:
feature_vectors_va_det

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.165079,0.137828,0.0,1.0,0.0,104.0,1.255411,1.804276,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.197987,0.158788,0.0,1.0,0.0,118.0,0.297709,1.515820,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.109244,0.097310,0.0,1.0,0.0,65.0,4.276488,2.505292,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.045276,0.043226,0.0,1.0,0.0,23.0,17.134378,4.374286,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.068041,0.063412,0.0,1.0,0.0,33.0,9.769979,3.430740,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.038055,0.036607,0.0,1.0,0.0,18.0,21.317339,4.828803,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.028481,0.027670,0.0,1.0,0.0,9.0,30.140427,5.669253,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.157702,0.132832,0.0,1.0,0.0,129.0,1.528313,1.878380,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.162562,0.136135,0.0,1.0,0.0,132.0,1.345633,1.829107,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [160]:
feature_vectors_va_det.to_parquet('../treino/va_det.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### Sica evoluçao mensal das consultas medicas hospitalares

In [161]:
sica_ev= pd.read_csv('sica_ev.csv')
sica_ev= sica_ev.squeeze()
sica_ev

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['2013/01/01', '2013/01/01', '2013/01/01', '20...
2    ['regiao de saude do alentejo', 'regiao de sau...
3    ['unidade local de saude do baixo alentejo, ep...
4    ['38.014123,-7.8721227', '40.6362453,-8.654371...
5    ['8443', '19526', '5940', '25665', '24763', '4...
Name: values, dtype: object

In [162]:
extract_features(
    "../treino/sica_ev_stats.csv",
    sica_ev
)
feature_vectors_sica_ev = pd.read_csv("../treino/sica_ev_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 29.41it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 27.03it/s]


In [163]:
feature_vectors_sica_ev[feature_vectors_sica_ev.columns[~feature_vectors_sica_ev.columns.str.contains(pat = 'par_vec|n_|word')]]

,col_entropy,frac_unique,frac_numcells,frac_textcells,avg_num_cells,std_num_cells,avg_text_cells,std_text_cells,avg_spec_cells,std_spec_cells,...,length-agg-min,length-agg-max,length-agg-median,length-agg-sum,length-agg-kurtosis,length-agg-skewness,none-agg-has,none-agg-percent,none-agg-num,none-agg-all
0,3.278950,0.014,0.542,0.000,0.542,0.498233,0.000,0.000000,0.102,0.302648,...,1.0,1.0,1.0,1000.0,-3.0,0.0,0.0,0.0,0.0,0.0
1,3.227259,0.014,0.570,0.000,0.570,0.495076,0.000,0.000000,0.077,0.266591,...,1.0,1.0,1.0,1000.0,-3.0,0.0,0.0,0.0,0.0,0.0
2,3.734642,0.018,0.000,0.715,0.000,0.000000,0.715,0.451414,0.026,0.159135,...,1.0,1.0,1.0,1000.0,-3.0,0.0,0.0,0.0,0.0,0.0
3,4.041474,0.025,0.000,0.779,0.000,0.000000,0.779,0.414920,0.042,0.200589,...,1.0,1.0,1.0,1000.0,-3.0,0.0,0.0,0.0,0.0,0.0
4,3.838714,0.015,0.669,0.000,0.669,0.470573,0.000,0.000000,0.162,0.368451,...,1.0,1.0,1.0,1000.0,-3.0,0.0,0.0,0.0,0.0,0.0
5,3.492041,0.013,0.569,0.000,0.569,0.495216,0.000,0.000000,0.101,0.301329,...,1.0,1.0,1.0,1000.0,-3.0,0.0,0.0,0.0,0.0,0.0


In [164]:
feature_vectors_sica_ev

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.181,0.148239,0.0,1.0,0.0,181.0,0.745863,1.657065,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.193,0.155751,0.0,1.0,0.0,193.0,0.420505,1.555797,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.047,0.044791,0.0,1.0,0.0,47.0,16.325914,4.280878,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.040,0.038400,0.0,1.0,0.0,40.0,20.041666,4.694855,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [165]:
feature_vectors_sica_ev.to_parquet('../treino/sica_ev.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### atendimentos em urgencia triagem manchester

In [166]:
tri_man= pd.read_csv('tri_man.csv')
tri_man= tri_man.squeeze()
tri_man

0     ['2013-01', '2013-01', '2013-01', '2013-01', '...
1     ['regiao de saude do alentejo', 'regiao de sau...
2     ['unidade local de saude do norte alentejano, ...
3     ['39.3003425,-7.4263845', '40.2804158,-7.49224...
4     ['35.0', '10.0', '28.0', '62.0', '127.0', '49....
5     ['880.0', '247.0', '679.0', '1218.0', '2403.0'...
6     ['2326.0', '2802.0', '2484.0', '7628.0', '8459...
7     ['4441.0', '1910.0', '3375.0', '11463.0', '468...
8     ['26.0', '15.0', '40.0', '129.0', '433.0', '15...
9     ['224.0', '149.0', '429.0', '404.0', '159.0', ...
10    ['25.0', '2099.0', '321.0', '1831.0', '8421.0'...
Name: values, dtype: object

In [167]:
extract_features(
    "../treino/tri_man_stats.csv",
    tri_man
)
feature_vectors_tri_man = pd.read_csv("../treino/tri_man_stats.csv", dtype=np.float32)

A extrair Features:  27%|█████████████████▏                                             | 3/11 [00:00<00:00, 27.52it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 28.49it/s]


In [168]:
feature_vectors_tri_man

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.187,0.152031,0.0,1.0,0.0,187.0,0.577606,1.605492,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.051,0.048399,0.0,1.0,0.0,51.0,14.661584,4.081861,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.122,0.107116,0.0,1.0,0.0,122.0,3.335674,2.309908,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.154,0.130284,0.0,1.0,0.0,154.0,1.675540,1.917170,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.125,0.109375,0.0,1.0,0.0,125.0,3.142857,2.267787,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.122,0.107116,0.0,1.0,0.0,122.0,3.335674,2.309908,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.150,0.127500,0.0,1.0,0.0,150.0,1.843137,1.960392,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.130,0.113100,0.0,1.0,0.0,130.0,2.841733,2.200394,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [169]:
feature_vectors_tri_man.to_parquet('../treino/tri_man.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### atividade de internamento hospitalar

In [170]:
ati_h= pd.read_csv('ati_h.csv')
ati_h= ati_h.squeeze()
ati_h

0    ['2015-01', '2015-01', '2015-01', '2015-01', '...
1    ['regiao de saude do algarve', 'regiao de saud...
2    ['centro hospitalar universitario do algarve,e...
3    ['37.0238973,-7.9281554', '39.7414812,-8.79827...
4    ['especialidade cirurgica', 'especialidade cir...
5    ['1135.0', '751.0', '219.0', '455.0', '1442.0'...
6    ['8418', '4336', '1334', '3094', '8474', '3375...
Name: values, dtype: object

In [171]:
extract_features(
    "../treino/ati_h_stats.csv",
    ati_h
)
feature_vectors_ati_h = pd.read_csv("../treino/ati_h_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 28.30it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 28.56it/s]


In [172]:
feature_vectors_ati_h

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.177,0.145671,0.0,1.0,0.0,177.0,0.864784,1.692567,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.050,0.047500,0.0,1.0,0.0,50.0,15.052631,4.129483,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.124,0.108624,0.0,1.0,0.0,124.0,3.206069,2.281681,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.047,0.044791,0.0,1.0,0.0,47.0,16.325914,4.280878,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [173]:
feature_vectors_ati_h.to_parquet('../treino/ati_h.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### cirurgias em ambulatorio

In [174]:
c_amb= pd.read_csv('c_amb.csv')
c_amb= c_amb.squeeze()
c_amb

0    ['2020-08', '2020-08', '2020-08', '2020-08', '...
1    ['regiao de saude lvt', 'regiao de saude lvt',...
2    ['centro hospitalar de lisboa ocidental, epe',...
3    ['38.708454,-9.216985', '38.7569771,-9.1460705...
4    ['1196', '0', '1573', '2300', '1141', '1547', ...
5    ['1591', '0', '1879', '2747', '1282', '1885', ...
6    ['75.2', '0.0', '83.7', '83.7', '89.0', '82.1'...
Name: values, dtype: object

In [175]:
extract_features(
    "../treino/c_amb_stats.csv",
    c_amb
)
feature_vectors_c_amb = pd.read_csv("../treino/c_amb_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 27.02it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 27.34it/s]


In [176]:
feature_vectors_c_amb

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.182,0.148876,0.0,1.0,0.0,182.0,0.716999,1.648332,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.056,0.052864,0.0,1.0,0.0,56.0,12.916465,3.862184,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.061,0.057279,0.0,1.0,0.0,61.0,11.458405,3.668570,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.046,0.043884,0.0,1.0,0.0,46.0,16.787348,4.334437,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.042,0.040236,0.0,1.0,0.0,42.0,18.853365,4.566548,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [177]:
feature_vectors_c_amb.to_parquet('../treino/c_amb.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### consultas em telemedicina

In [178]:
telemed= pd.read_csv('telemed.csv')
telemed= telemed.squeeze()
telemed

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['hospital espirito santo de evora, epe', 'uni...
3    ['38.5684861,-7.9032848', '38.014123,-7.872122...
4    ['133', '1', '68', '19', '74', '172', '48', '3...
Name: values, dtype: object

In [179]:
extract_features(
    "../treino/telemed_stats.csv",
    telemed
)
feature_vectors_telemed = pd.read_csv("../treino/telemed_stats.csv", dtype=np.float32)

A extrair Features:  60%|██████████████████████████████████████▍                         | 3/5 [00:00<00:00, 25.00it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 23.92it/s]


In [180]:
feature_vectors_telemed

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.193,0.155751,0.0,1.0,0.0,193.0,0.420505,1.555797,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.038,0.036556,0.0,1.0,0.0,38.0,21.355291,4.832731,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.049,0.046599,0.0,1.0,0.0,49.0,15.459688,4.178479,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [181]:
feature_vectors_telemed.to_parquet('../treino/telemed.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### consultas em tempo real

In [182]:
temp_r= pd.read_csv('temp_r.csv')
temp_r= temp_r.squeeze()
temp_r

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['hospital espirito santo de evora, epe', 'cen...
3    ['38.5684861,-7.9032848', '40.2804158,-7.49224...
4    ['531', '336', '157', '551', '774', '955', '14...
5    ['1211', '338', '167', '600', '1370', '1184', ...
6    ['43.8', '99.4', '94.0', '91.8', '56.5', '80.7...
Name: values, dtype: object

In [183]:
extract_features(
    "../treino/temp_r_stats.csv",
    temp_r
)
feature_vectors_temp_r = pd.read_csv("../treino/temp_r_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 25.64it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 26.82it/s]


In [184]:
feature_vectors_temp_r

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.183,0.149511,0.0,1.0,0.0,183.0,0.688471,1.639656,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.049,0.046599,0.0,1.0,0.0,49.0,15.459688,4.178479,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.036,0.034704,0.0,1.0,0.0,36.0,22.815123,4.981478,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.021,0.020559,0.0,1.0,0.0,21.0,42.640499,6.681355,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [185]:
feature_vectors_temp_r.to_parquet('../treino/temp_r.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### consumo

In [186]:
cons= pd.read_csv('cons.csv')
cons= cons.squeeze()
cons

0    ['2010-12', '2010-12', '2010-12', '2011-12', '...
1    ['regiao de saude lvt', 'regiao de saude do al...
2    ['centro hospitalar medio tejo, epe', 'unidade...
3    ['39.478072,-8.5404429', '38.014123,-7.8721227...
4    ['total', 'total', 'total', 'total', 'total', ...
5    ['9676.0', '2559.0', '1889.0', '23914.0', '581...
Name: values, dtype: object

In [187]:
extract_features(
    "../treino/cons_stats.csv",
    cons
)
feature_vectors_cons = pd.read_csv("../treino/cons_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 26.32it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 28.57it/s]


In [188]:
feature_vectors_cons

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.166,0.138444,0.0,1.0,0.0,166.0,1.223137,1.795310,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.040,0.038400,0.0,1.0,0.0,40.0,20.041666,4.694855,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.135,0.116775,0.0,1.0,0.0,135.0,2.563477,2.136230,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [189]:
feature_vectors_cons.to_parquet('../treino/cons.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### fraturas na anca cirurgias nas primeiras 48h

In [190]:
frat_anc= pd.read_csv('frat_anc.csv')
frat_anc= frat_anc.squeeze()
frat_anc

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['hospital espirito santo de evora, epe', 'cen...
3    ['38.5684861,-7.9032848', '40.297919,-8.770780...
4    ['2', '0', '0', '6', '5', '21', '9', '7', '14'...
5    ['25', '0', '0', '34', '20', '31', '25', '12',...
6    ['8.0', '0.0', '0.0', '17.65', '25.0', '67.74'...
Name: values, dtype: object

In [191]:
extract_features(
    "../treino/frat_anc_stats.csv",
    frat_anc
)
feature_vectors_frat_anc = pd.read_csv("../treino/frat_anc_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 26.79it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 27.66it/s]


In [192]:
feature_vectors_frat_anc

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.189,0.153279,0.0,1.0,0.0,189.0,0.524051,1.588726,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.039,0.037479,0.0,1.0,0.0,39.0,20.681608,4.762521,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.039,0.037479,0.0,1.0,0.0,39.0,20.681608,4.762521,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.054,0.051084,0.0,1.0,0.0,54.0,13.575601,3.946594,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.067,0.062511,0.0,1.0,0.0,67.0,9.997185,3.463695,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [193]:
feature_vectors_frat_anc.to_parquet('../treino/frat_anc.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### incritos lic dentro tmrg

In [194]:
inc_lic= pd.read_csv('inc_lic.csv')
inc_lic= inc_lic.squeeze()
inc_lic

0    ['2013-06', '2013-06', '2013-06', '2013-06', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['unidade local de saude do litoral alentejano...
3    ['38.0162314,-8.6956768', '40.2191523,-8.40982...
4    ['1036', '12903', '180', '362', '657', '3167',...
5    ['1051', '17000', '186', '377', '772', '3863',...
6    ['98.6', '75.9', '96.8', '96.0', '85.1', '82.0...
Name: values, dtype: object

In [195]:
extract_features(
    "../treino/inc_lic_stats.csv",
    inc_lic
)
feature_vectors_inc_lic = pd.read_csv("../treino/inc_lic_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 27.26it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 26.92it/s]


In [196]:
feature_vectors_inc_lic

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.192,0.155136,0.0,1.0,0.0,192.0,0.445957,1.563956,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.040,0.038400,0.0,1.0,0.0,40.0,20.041666,4.694855,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.043,0.041151,0.0,1.0,0.0,43.0,18.300747,4.505635,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.035,0.033775,0.0,1.0,0.0,35.0,23.607698,5.060405,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [197]:
feature_vectors_inc_lic.to_parquet('../treino/inc_lic.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### intervençoes cirurgicas

In [198]:
int_cir= pd.read_csv('int_cir.csv')
int_cir= int_cir.squeeze()
int_cir

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['hospital espirito santo de evora, epe', 'hos...
3    ['38.5684861,-7.9032848', '40.3470015,-8.59334...
4    ['1257', '95', '464', '541', '514', '520', '19...
5    ['538.0', '0.0', '293.0', '187.0', '229.0', '2...
6    ['719', '95', '171', '354', '285', '258', '111...
7    ['160.0', '0.0', '0.0', '202.0', '96.0', '142....
Name: values, dtype: object

In [199]:
extract_features(
    "../treino/int_cir_stats.csv",
    int_cir
)
feature_vectors_int_cir = pd.read_csv("../treino/int_cir_stats.csv", dtype=np.float32)

A extrair Features:  25%|████████████████                                                | 2/8 [00:00<00:00, 17.86it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 23.32it/s]


In [200]:
feature_vectors_int_cir

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.196,0.157584,0.0,1.0,0.0,196.0,0.345822,1.531608,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.056,0.052864,0.0,1.0,0.0,56.0,12.916465,3.862184,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.122,0.107116,0.0,1.0,0.0,122.0,3.335674,2.309908,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.035,0.033775,0.0,1.0,0.0,35.0,23.607698,5.060405,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.146,0.124684,0.0,1.0,0.0,146.0,2.020275,2.005062,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [201]:
feature_vectors_int_cir.to_parquet('../treino/int_cir.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### nº utentes inscritos na area do cidadao

In [202]:
ut_insc= pd.read_csv('ut_insc.csv')
ut_insc= ut_insc.squeeze()
ut_insc

0    ['2019-04', '2019-04', '2019-04', '2019-04', '...
1    ['regiao de saude do norte', 'regiao de saude ...
2    ['polo canedo', 'polo escapaes', 'usf argoncil...
3    ['631.0', '2285.0', '1928.0', '2131.0', '1754....
4    ['14.9313771888', '54.6389287422', '25.4655923...
Name: values, dtype: object

In [203]:
extract_features(
    "../treino/ut_insc_stats.csv",
    ut_insc
)
feature_vectors_ut_insc = pd.read_csv("../treino/ut_insc_stats.csv", dtype=np.float32)

A extrair Features:   0%|                                                                        | 0/5 [00:00<?, ?it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 23.04it/s]


In [204]:
feature_vectors_ut_insc

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.177,0.145671,0.0,1.0,0.0,177.0,0.864784,1.692567,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.152,0.128896,0.0,1.0,0.0,152.0,1.758193,1.938606,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.107,0.095551,0.0,1.0,0.0,107.0,4.465615,2.542758,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [205]:
feature_vectors_ut_insc.to_parquet('../treino/ut_insc.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### ocupaçao do internamento

In [206]:
oc_int= pd.read_csv('oc_int.csv')
oc_int= oc_int.squeeze()
oc_int

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do alentejo', 'regiao de sau...
2    ['unidade local de saude do baixo alentejo, ep...
3    ['38.014123,-7.8721227', '40.2191523,-8.409821...
4    ['5753', '49170', '15016', '0', '16267', '6737...
5    ['229.0', '1920.0', '545.0', '0.0', '0.0', '12...
6    ['82.5', '84.1', '90.5', '0.0', '0.0', '90.0',...
Name: values, dtype: object

In [207]:
extract_features(
    "../treino/oc_int_stats.csv",
    oc_int
)
feature_vectors_oc_int = pd.read_csv("../treino/oc_int_stats.csv", dtype=np.float32)

A extrair Features:  43%|███████████████████████████▍                                    | 3/7 [00:00<00:00, 27.03it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 27.56it/s]


In [208]:
feature_vectors_oc_int

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.201,0.160599,0.0,1.0,0.0,201.0,0.226689,1.492209,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.052,0.049296,0.0,1.0,0.0,52.0,14.285622,4.035545,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.048,0.045696,0.0,1.0,0.0,48.0,15.883754,4.228919,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.141,0.121119,0.0,1.0,0.0,141.0,2.256343,2.063091,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.033,0.031911,0.0,1.0,0.0,33.0,25.337156,5.228495,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [209]:
feature_vectors_oc_int.to_parquet('../treino/oc_int.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### rncci episodios

In [210]:
rncci= pd.read_csv('rncci.csv')
rncci= rncci.squeeze()
rncci

0    ['2022-04-26', '2022-04-26', '2022-04-26', '20...
1    ['alentejo', 'alentejo', 'algarve', 'centro', ...
2    ['uc', 'umdr', 'ecci', 'uc', 'ra', 'rama', 'uc...
3    ['48', '37', '2', '24', '1', '22', '213', '2',...
Name: values, dtype: object

In [211]:
extract_features(
    "../treino/rncci_stats.csv",
    rncci
)
feature_vectors_rncci = pd.read_csv("../treino/rncci_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 27.21it/s]

Exporting 1588 column features


In [212]:
feature_vectors_rncci

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.170,0.141100,0.0,1.0,0.0,170.0,1.087172,1.757035,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.015,0.014775,0.0,1.0,0.0,15.0,61.681896,7.980093,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [213]:
feature_vectors_rncci.to_parquet('../treino/rncci.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### taxa de mortalidade por avc isquemico e hemorragico

In [214]:
tax_avc= pd.read_csv('tax_avc.csv')
tax_avc= tax_avc.squeeze()
tax_avc

0    ['2013-01', '2013-01', '2013-01', '2013-01', '...
1    ['regiao de saude do centro', 'regiao de saude...
2    ['centro hospitalar universitario cova da beir...
3    ['40.2804158,-7.4922407', '40.2191523,-8.40982...
4    ['50.0', '15.15', '28.57', '33.33', '60.0', '1...
5    ['5.0', '12.8', '19.23', '7.69', '12.9', '11.6...
Name: values, dtype: object

In [215]:
extract_features(
    "../treino/tax_avc_stats.csv",
    tax_avc
)
feature_vectors_tax_avc = pd.read_csv("../treino/tax_avc_stats.csv", dtype=np.float32)

A extrair Features:  50%|████████████████████████████████                                | 3/6 [00:00<00:00, 26.31it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 26.91it/s]


In [216]:
feature_vectors_tax_avc

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.186,0.151404,0.0,1.0,0.0,186.0,0.604845,1.613953,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.044,0.042064,0.0,1.0,0.0,44.0,17.773298,4.446718,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.066,0.061644,0.0,1.0,0.0,66.0,10.222179,3.496023,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [217]:
feature_vectors_tax_avc.to_parquet('../treino/tax_avc.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### teleconsultas atraves da plataforma de telesaude

In [218]:
tel_sad= pd.read_csv('tel_sad.csv')
tel_sad= tel_sad.squeeze()
tel_sad

0    ['2016-10', '2017-02', '2017-10', '2017-12', '...
1    ['15.0', '175.0', '111.0', '55.0', '45.0', '59...
2    ['22.0', '423.0', '288.0', '122.0', '97.0', '1...
3    ['14', '140', '98', '52', '43', '55', '57', '4...
4    ['53.0', '2749.0', '2115.0', '1265.0', '933.0'...
5    ['38.0', '3535.0', '2442.0', '1657.0', '1291.0...
Name: values, dtype: object

In [219]:
extract_features(
    "../treino/tel_sad_stats.csv",
    tel_sad
)
feature_vectors_tel_sad = pd.read_csv("../treino/tel_sad_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 56.07it/s]

Exporting 1588 column features


In [220]:
feature_vectors_tel_sad

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.177778,0.146173,0.0,1.0,0.0,88.0,0.841216,1.685591,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.131720,0.114370,0.0,1.0,0.0,49.0,2.743540,2.177967,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,1.0,0.0,0.120907,0.106288,0.0,1.0,0.0,48.0,3.408369,2.325590,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.021739,0.021267,0.0,1.0,0.0,6.0,41.022221,6.559133,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.143519,0.122921,0.0,1.0,0.0,62.0,2.135309,2.033546,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.135011,0.116783,0.0,1.0,0.0,59.0,2.562864,2.136086,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [221]:
feature_vectors_tel_sad.to_parquet('../treino/tel_sad.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### trabalhadores por grupo profissional

In [222]:
trab_g= pd.read_csv('trab_g.csv')
trab_g= trab_g.squeeze()
trab_g

0     ['2014-12', '2015-02', '2015-03', '2015-04', '...
1     ['servicos centrais', 'regiao de saude do alen...
2     ['acao governativa', 'administracao regional d...
3     ['38.7362218,-9.1463126', '38.565715,-7.91162'...
4     ['0.0', '108.0', '902.0', '0.0', '0.0', '221.0...
5     ['0.0', '29.0', '346.0', '0.0', '0.0', '84.0',...
6     ['0.0', '178.0', '1147.0', '0.0', '0.0', '410....
7     ['0.0', '18.0', '75.0', '0.0', '0.0', '37.0', ...
8     ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
9     ['0.0', '24.0', '152.0', '0.0', '0.0', '136.0'...
10    ['6', '179', '1002', '6', '6', '306', '1879', ...
11    ['6', '98', '534', '6', '6', '253', '880', '6'...
12    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
13    ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0....
14    ['30', '85', '233', '32', '31', '112', '416', ...
15    ['42', '719', '4391', '44', '43', '1559', '876...
Name: values, dtype: object

In [223]:
extract_features(
    "../treino/trab_g_stats.csv",
    trab_g
)
feature_vectors_trab_g = pd.read_csv("../treino/trab_g_stats.csv", dtype=np.float32)

A extrair Features:  19%|███████████▊                                                   | 3/16 [00:00<00:00, 26.09it/s]

Exporting 1588 column features


A extrair Features: 100%|██████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 29.63it/s]


In [224]:
feature_vectors_trab_g

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.191,0.154519,0.0,1.0,0.0,191.0,0.471696,1.572163,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,1.0,0.0,0.163,0.136431,0.0,1.0,0.0,163.0,1.329713,1.824750,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.186,0.151404,0.0,1.0,0.0,186.0,0.604845,1.613953,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.159,0.133719,0.0,1.0,0.0,159.0,1.478369,1.865039,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.178,0.146316,0.0,1.0,0.0,178.0,0.834523,1.683604,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.246,0.185484,0.0,1.0,0.0,246.0,-0.608699,1.179534,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,1.0,0.0,0.141,0.121119,0.0,1.0,0.0,141.0,2.256343,2.063091,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [225]:
feature_vectors_trab_g.to_parquet('../treino/trab_g.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### Dados sinteticos atualizado

In [226]:
dad_s2= pd.read_csv('dados_sinteticos2.csv')
dad_s2= dad_s2.squeeze()
dad_s2

0    ['+351 935770576', '+351 917381362', '+351 912...
1    ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', ...
2    [' portugal', ' portugal', ' portugal', ' port...
3    ['lisbon', 'espinho', 'matosinhos', 'lisbon', ...
4    ['single', 'single', 'married', 'in a relation...
5    ['estudante', 'profissional liberal de informa...
Name: values, dtype: object

In [227]:
extract_features(
    "../treino/dad_s_stats2.csv",
    dad_s2
)
feature_vectors_dad_s2 = pd.read_csv("../treino/dad_s_stats2.csv", dtype=np.float32)

A extrair Features:   0%|                                                                        | 0/6 [00:00<?, ?it/s]

Exporting 1588 column features


A extrair Features: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 26.54it/s]


In [228]:
feature_vectors_dad_s2

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.037,0.035631,0.0,1.0,0.0,37.0,22.065449,4.905655,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.001,0.000999,0.0,1.0,0.0,1.0,995.000977,31.575323,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [229]:
feature_vectors_dad_s2.to_parquet('../treino/dad_s2.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


### dados novos

In [237]:
dad_nv= pd.read_csv('dados_novos.csv')
dad_nv= dad_nv.squeeze()
dad_nv

0    ['1', '2', '3', '4', '5', '5', '5', '5', '5', ...
1    ['0.0', '1.0', '11.0', '111.0', '111.1', '111....
2    ['profissoes das forcas armadas', 'oficiais da...
Name: values, dtype: object

In [238]:
extract_features(
    "../treino/dad_novos_stats.csv",
    dad_nv
)
feature_vectors_dad_nv = pd.read_csv("../treino/dad_novos_stats.csv", dtype=np.float32)

A extrair Features: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 15.70it/s]

Exporting 1588 column features


In [240]:
feature_vectors_dad_nv

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,1.0,0.0,0.086,0.078604,0.0,1.0,0.0,86.0,6.721999,2.953303,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [239]:
feature_vectors_dad_nv.to_parquet('../treino/dad_nv.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')

Save parquet process


---------------------------

# juntar td os feature vectors num so

In [241]:
#pieces = (df1,df2,df3,df4,df5,df6,df7,df8)
#df_final = pd.concat(pieces, ignore_index = True)
#all_dfs = [df1, df2, df3]
#pd.concat(all_dfs).reset_index(drop=True)


pieces= [feature_vectors_anti,feature_vectors_cert_obit,feature_vectors_col_org,feature_vectors_dad_org,
        feature_vectors_dad_sang,feature_vectors_dad_vf,feature_vectors_ddc_ob,feature_vectors_ep_int,
        feature_vectors_casos_g,feature_vectors_inc_sp,feature_vectors_mcu, feature_vectors_an_gen,
        feature_vectors_res,feature_vectors_sa_mc,feature_vectors_tax_c,feature_vectors_tax_d,
        feature_vectors_tansp,feature_vectors_vac,feature_vectors_ant_ca, feature_vectors_ant_ce,
        feature_vectors_dm_dm,feature_vectors_diss_t,
        feature_vectors_ev_ic,feature_vectors_freq_csg,
        feature_vectors_map_ccu,feature_vectors_mal,feature_vectors_nnd_rn,
        feature_vectors_pc,feature_vectors_rtv,
        feature_vectors_amos,feature_vectors_datas,feature_vectors_data_c,feature_vectors_rt, feature_vectors_vaci,
        feature_vectors_va_det,feature_vectors_sica_ev,feature_vectors_tri_man,feature_vectors_ati_h,feature_vectors_c_amb,
        feature_vectors_telemed,feature_vectors_temp_r,feature_vectors_cons,feature_vectors_frat_anc,feature_vectors_inc_lic,
        feature_vectors_int_cir,feature_vectors_ut_insc,feature_vectors_oc_int,feature_vectors_rncci,feature_vectors_tax_avc,
        feature_vectors_tel_sad,feature_vectors_trab_g,feature_vectors_dad_s2,feature_vectors_dad_nv]

feature_vectors_final=pd.concat(pieces).reset_index(drop=True)


In [242]:
feature_vectors_final

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.205000,0.162975,0.0,1.0,0.0,205.0,0.135910,1.461475,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.045000,0.042975,0.0,1.0,0.0,45.0,17.269342,4.389686,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.029000,0.028159,0.0,1.0,0.0,29.0,29.512625,5.613611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.040000,0.038400,0.0,1.0,0.0,40.0,20.041666,4.694855,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.101000,0.090799,0.0,1.0,0.0,101.0,5.013337,2.648271,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.187000,0.152031,0.0,1.0,0.0,187.0,0.577606,1.605492,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


In [243]:
feature_vectors_final.shape

(382, 1588)

In [244]:
a= pd.read_csv('anti_labels.csv')
a1= pd.read_csv('cert_obit_labels.csv')
a2= pd.read_csv('col_org_labels.csv')
a3= pd.read_csv('dad_org_labels.csv')
a4= pd.read_csv('dad_sang_labels.csv')
a5= pd.read_csv('dad_vf_labels.csv')
a6= pd.read_csv('ddc_ob_labels.csv')
a7= pd.read_csv('ep_int_labels.csv')
a8= pd.read_csv('casos_g_labels.csv')
a9= pd.read_csv('inc_sp_labels.csv')
a10= pd.read_csv('mcu_labels.csv')
a11= pd.read_csv('an_gen_labels.csv')
a12= pd.read_csv('res_labels.csv')
a13= pd.read_csv('sa_mc_labels.csv')
a14= pd.read_csv('tax_c_labels.csv')
a15= pd.read_csv('tax_d_labels.csv')
a16= pd.read_csv('tansp_labels.csv')
a17= pd.read_csv('vac_labels.csv')
a18= pd.read_csv('ant_ca_labels.csv')
a19= pd.read_csv('ant_ce_labels.csv')
#a20= pd.read_csv('cert_obi_labels.csv')
#a21= pd.read_csv('dad_vef_labels.csv')
a22= pd.read_csv('dm_dm_labels.csv')
a23= pd.read_csv('diss_t_labels.csv')
#a24= pd.read_csv('ep_interdm_labels.csv')
a25= pd.read_csv('ev_ic_labels.csv')
#a26= pd.read_csv('freq_cg_labels.csv')
a27= pd.read_csv('freq_csg_labels.csv')
#a28= pd.read_csv('inc_sg_labels.csv')
a29= pd.read_csv('map_ccu_labels.csv')
a30= pd.read_csv('mal_labels.csv')
a31= pd.read_csv('nnd_rn_labels.csv')
a32= pd.read_csv('pc_labels.csv')
a33= pd.read_csv('rtv_labels.csv')
#a34= pd.read_csv('tinc_d_labels.csv')
#a35= pd.read_csv('trp_labels.csv')
#a36= pd.read_csv('va_labels.csv')
a37= pd.read_csv('amos_labels.csv')
a38= pd.read_csv('datas_labels.csv')
a39= pd.read_csv('data_c_labels.csv')
a40= pd.read_csv('rt_labels.csv')
a41= pd.read_csv('vaci_labels.csv')
a42= pd.read_csv('va_det_labels.csv')
a43= pd.read_csv('sica_ev_labels.csv')
a44= pd.read_csv('tri_man_labels.csv')
a45= pd.read_csv('ati_h_labels.csv')
a46= pd.read_csv('c_amb_labels.csv')
a47= pd.read_csv('telemed_labels.csv')
a48= pd.read_csv('temp_r_labels.csv')
a49= pd.read_csv('cons_labels.csv')
a50= pd.read_csv('frat_anc_labels.csv')
a51= pd.read_csv('inc_lic_labels.csv')
a52= pd.read_csv('int_cir_labels.csv')
a53= pd.read_csv('ut_insc_labels.csv')
a54= pd.read_csv('oc_int_labels.csv')
a55= pd.read_csv('rncci_labels.csv')
a56= pd.read_csv('tax_avc_labels.csv')
a57= pd.read_csv('tel_sad_labels.csv')
a58= pd.read_csv('trab_g_labels.csv')
a59= pd.read_csv('dados_sinteticos_labels2.csv')
a60= pd.read_csv('dados_novos_labels.csv')




labels=[a,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a22,a23,a25,a27,a29,a30,a31,
       a32,a33,a37,a38,a39,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,a60]



labels_final=pd.concat(labels).reset_index(drop=True)





In [245]:
labels_final

,labels
0,periodo
1,ars
2,hospital
3,localizacao geografica
4,grupo hospitalar
5,unidades antibioticos
6,unidades totais
7,peso
8,periodo
9,instituicao de saude


In [246]:
labels_final.shape

(382, 1)

## transformar para ficheiro parquet

In [247]:
#feature_vectors
feature_vectors_final.to_parquet('../sherlock-project/feature_vectors_final3_novo.parquet', engine='pyarrow', compression='snappy')

#labels
labels_final.to_parquet('../sherlock-project/labels_final3_novo.parquet', engine='pyarrow', compression='snappy')


# Nova abordagem - repetir os nossos dados em 10x -> da erro na previsao devido a entropia

### labels

In [265]:
labels_nova=pd.concat([labels_final]*10, ignore_index=True)

In [266]:
labels_nova

,labels
0,periodo
1,ars
2,hospital
3,localizacao geografica
4,grupo hospitalar
5,unidades antibioticos
6,unidades totais
7,peso
8,periodo
9,instituicao de saude


### features

In [267]:
feature_vectors_nova=pd.concat([feature_vectors_final]*10, ignore_index=True)

In [268]:
feature_vectors_nova

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,1.0,0.0,0.205000,0.162975,0.0,1.0,0.0,205.0,0.135910,1.461475,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
3,1.0,0.0,0.045000,0.042975,0.0,1.0,0.0,45.0,17.269342,4.389686,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
5,1.0,0.0,0.029000,0.028159,0.0,1.0,0.0,29.0,29.512625,5.613611,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
6,1.0,0.0,0.040000,0.038400,0.0,1.0,0.0,40.0,20.041666,4.694855,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
7,1.0,0.0,0.101000,0.090799,0.0,1.0,0.0,101.0,5.013337,2.648271,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
8,1.0,0.0,0.187000,0.152031,0.0,1.0,0.0,187.0,0.577606,1.605492,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787
9,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.001093,-0.00019,-0.000603,0.000873,-0.001167,0.001147,-0.000362,-0.000358,-0.001209,-0.000787


### guardar novos dados para ficheiro .parquet

In [269]:
#feature_vectors
feature_vectors_nova.to_parquet('../sherlock-project/feature_vectors_novas.parquet', engine='pyarrow', compression='snappy')

#labels
labels_nova.to_parquet('../sherlock-project/labels_novas.parquet', engine='pyarrow', compression='snappy')


------------------------------------------------------------------

# seleccionar conjunto de teste

aqui tenho de pegar em tds os databases originais e guardar td num so e depois seleccionar os de conjunto de teste. Next tenho de converter para o formato parquet definido.
Feito isso podemos comparar aos ficheiros de teste

a= pd.read_csv('anti1.csv')
a1= pd.read_csv('cert_obit1.csv')
a2= pd.read_csv('col_org1.csv')
a3= pd.read_csv('dad_org1.csv')
a4= pd.read_csv('dad_sang1.csv')
a5= pd.read_csv('dad_vf1.csv')
a6= pd.read_csv('ddc_ob1.csv')
a7= pd.read_csv('ep_int1.csv')
a8= pd.read_csv('casos_g1.csv')
a9= pd.read_csv('inc_sp1.csv')
a10= pd.read_csv('mcu1.csv')
a11= pd.read_csv('an_gen1.csv')
a12= pd.read_csv('res1.csv')
a13= pd.read_csv('sa_mc1.csv')
a14= pd.read_csv('tax_c1.csv')
a15= pd.read_csv('tax_d1.csv')
a16= pd.read_csv('tansp1.csv')
a17= pd.read_csv('vac1.csv')
a18= pd.read_csv('ant_ca1.csv')
a19= pd.read_csv('ant_ce1.csv')
a22= pd.read_csv('dm_dm1.csv')
a23= pd.read_csv('diss_t1.csv')
a25= pd.read_csv('ev_ic1.csv')
a27= pd.read_csv('freq_csg1.csv')
a29= pd.read_csv('map_ccu1.csv')
a30= pd.read_csv('mal1.csv')
a31= pd.read_csv('nnd_rn1.csv')
a32= pd.read_csv('pc1.csv')
a33= pd.read_csv('rtv1.csv')
a37= pd.read_csv('amos1.csv')
a38= pd.read_csv('datas1.csv')
a39= pd.read_csv('data_c1.csv')
a40= pd.read_csv('rt1.csv')
a41= pd.read_csv('vaci1.csv')
a42= pd.read_csv('va_det1.csv')
a43= pd.read_csv('sica_ev1.csv')
a44= pd.read_csv('tri_man1.csv')
a45= pd.read_csv('ati_h1.csv')
a46= pd.read_csv('c_amb1.csv')
a47= pd.read_csv('telemed1.csv')
a48= pd.read_csv('temp_r1.csv')
a49= pd.read_csv('cons1.csv')
a50= pd.read_csv('frat_anc1.csv')
a51= pd.read_csv('inc_lic1.csv')
a52= pd.read_csv('int_cir1.csv')
a53= pd.read_csv('ut_insc1.csv')
a54= pd.read_csv('oc_int1.csv')
a55= pd.read_csv('rncci1.csv')
a56= pd.read_csv('tax_avc1.csv')
a57= pd.read_csv('tel_sad1.csv')
a58= pd.read_csv('trab_g1.csv')

dt=[a,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a22,a23,a25,a27,a29,a30,a31,
       a32,a33,a37,a38,a39,a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58]



dt_final=pd.concat(dt,axis=1).reset_index(drop=True)





dt_final

dt_final.isnull().sum()

dt_final = dt_final.astype(str)

dt_final= dt_final.T
for col in dt_final.columns:
    dt_final[col] = dt_final[col].apply(unidecode)
    dt_final[col]=dt_final[col].astype(str).str.lower()

dt_final= dt_final.T
dt_final

#dt_final.columns = [x.apply(unidecode) for x in df.columns]

#dt_final.rename(columns=unidecode.unidecode)  
dt_final.columns = [unidecode.unidecode(col) for col in dt_final.columns]
dt_final.columns= dt_final.columns.astype(str).str.lower()
#for col in dt_final.columns:
 #   col= col.apply(unidecode)
  #  col= col.astype(str).str.lower()



dt_final



dt_final= dt_final[['virus da gripe dominante', 'no su t.m. azul', 'periodo',
       'pessoas reforco', 'pessoas inoculadas', 'ars',
       'no su t.m. laranja', 'periodo',
       'nr indiciados nao toxicodependentes', 'rt', 'pessoas gripe',
       'total de participantes', 'localizacao geografica',
       'utentes novos',
       'recem-nascidos c/ 1+ domicilio enf. < 15 dias de vida',
       'instituicao', 'periodo', 'pessoas vacinadas completamente',
       'rinovirus humano', 'cod. acesso hosp. grupo',
       'nr virus respiratorios detetados', 'periodo',
       'vacinacao iniciada 05-11', 'periodo', 'utentes readmitidos',
       'instituicao', 'orgao', 'indice icaro', 'unidades totais', 'ars',
       'periodo', 'rt', 'regiao', 'doses', 'localizacao geografica',
       'periodo', 'mortalidade mensal', 'dadores regulares', 'periodo',
       'virus da gripe b (linhagem yamagata)',
       'anomalias congenitas do aparelho urinario',
       'no orgaos transplantados', 'nr total de dadores de orgaos',
       'no primeiras ce prestadas dentro do tmrg',
       'nr reg. nascer utente', 'unidades antibioticos',
       'recem-nascidos c/ cons. med. vigil. ate 28 dias vida', 'periodo',
       'peso', 'anomalias congenitas dos olhos, ouvido, face e pescoco',
       '% 1as consultas realizadas em tempo adequado', 'sintoma',
       'vigilancia', 'media de participantes por minutos de sessao pds',
       'periodo', 'no su t.m. verde', 'no su t.m. amarela', 'periodo',
       'doses1 %', 'localizacao geografica', 'parainfluenza virus',
       'outras anomalias congenitas', 'no su t.m. vermelha',
       'aces hosp. grupo', 'desvio padrao da idade dos utentes novos',
       'semana', 'regiao', 'periodo', 'doses1', 'total geral',
       'taxa de incidencia de gripe semanal (100.000 hab)', 'periodo',
       'regiao', 'regiao', 'periodo',
       'taxa anual de ocupacao em internamento', 'peso', 'n_confirmados',
       'idade media dos utentes novos',
       'nomenclatura portuguesa do dispositivo medico (npdm)',
       'no primeiras ce realizadas com registo no cth',
       'localizacao geografica', 'regiao', 'tipo instituicao', 'periodo',
       'regiao', 'no utentes inscritos na area do cidadao', 'periodo',
       'virus da gripe a(h1)pdm09', 'localizacao geografica', 'uls grupo',
       'cod. nivel acesso', 'instituicao', 'nr cesarianas', 'periodo',
       'doses desconhecidas', 'entidade', 'recuperados',
       'localizacao geografica', 'cod. centro custo', 'periodo',
       'des. area atuacao princ. serv.', 'localizacao geografica', 'lab',
       'orgao', 'no de doentes em le', 'instituicao',
       'tipo uf serv. separt.', 'periodo', 'regiao',
       'total dadores vivos',
       'consultas dos test. vit. portal do profissional',
       'nr dispositivos medicos',
       'utentes inscritos com a vacina da gripe prescrita ou efetuada nos ultimos 12 meses',
       'des. instituicao', 'mortalidade avc isquemico 30 dias',
       'proporcao idosos ou doenca cronica, c/ vacina da gripe',
       'localizacao geografica', 'periodo', 'periodo', 'periodo',
       'periodo', 'localizacao geografica', 'ars', 'no s/ t.m.',
       'grupo sanguineo', 'ddd consumidas de carbapenemes',
       'tecnicos superiores', 'pessoas vacinadas parcialmente',
       'localizacao geografica', 'periodo', 'localizacao geografica',
       'nr casos gripe testados', 'medicos internos', 'regiao',
       'instituicao', 'doenca de declaracao obrigatoria', 'periodo',
       'sexo', 'ordem', 'sexo feminino', 'regiao', 'concelhos',
       'no ep. cir. amb. c/ proc. ambulatorizaveis',
       'no ep. cir. amb e int c/ proc. ambulatorizaveis',
       'tecnicos superiores de saude', 'suspeitos', 'ars',
       'proporcao criancas 2 anos, c/ pnv cumprido ate 2 anos',
       'virus da gripe', 'instituicao', 'uf serv. depart. hier. loc.',
       'no intervencoes cirurgicas de ambulatorio',
       'anomalias congenitas do aparelho genital', 'periodo',
       'media sessoes pds live', 'doses2 %',
       'cod. mes ini. mapeam. cc. hier. loc.',
       'nr dadores por milhao de habitantes (pmh)', 'instituicao',
       'virus da gripe b', 'entidade',
       'no intervencoes cirurgicas programadas', 'periodo',
       'vacinas reforco e gripe', 'assistentes operacionais',
       'des. centro custo', 'dias de internamento',
       'mortalidade avc hemorragico 30 dias',
       'nr processos de contraordenacao', 'entidade', 'entidade',
       'pessoas vacinadas completamente %', 'periodo', 'grupo etario',
       'vacinacao completa 05-11', 'nr casos', 'medicos s/ internos',
       'periodo', 'no casos sindroma gripal testados',
       'test. vit. de utentes c/ nacionalidade portuguesa',
       'ddd consumidas dos restantes ant.', 'localizacao geografica',
       'jovens com pnv totalmente cumprido na data do 14o aniversario',
       'cod. area atuacao princ. serv.',
       'no ep. utentes c/ cir. real. nas 1as 48h apos admissao',
       'pessoas vacinadas parcialmente %']]


dt_final= dt_final.astype(str)

dt_final.head()

dt_final.shape

dt_final.to_csv("dt_final.csv",index=False)

from collections import defaultdict

dt = defaultdict(list)

for i in dt_final.columns:
    dt[i]=dt_final[i].values

new_v= [i for i in dt.values()]

df = pd.DataFrame(new_v)

    

df = df.astype(str)

da=df.values.tolist()

dataframe = pd.DataFrame([da])
dataframe=dataframe.T

dataframe.columns=['values']
#dataframe['values']= dataframe['values'].str.lower()
#dataframe.to_parquet('dt_final.parquet',compression='snappy',index=None)
#dataframe


dataframe

dt_final.to_csv("dt_final.csv")

dt_final.info(memory_usage="deep")

dt_final.iloc[:,187:]

______________________________________________

# Dados Sintéticos

dad_s= pd.read_csv('dados_sinteticos.csv')
dad_s= dad_s.squeeze()
dad_s

extract_features(
    "../treino/dad_s_stats.csv",
    dad_s
)
feature_vectors_dad_s = pd.read_csv("../treino/dad_s_stats.csv", dtype=np.float32)

feature_vectors_dad_s

feature_vectors_dad_s.to_parquet('../treino/dad_s.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process')